# Mistral OCR

In [6]:
import base64
import json
import os

# import re
from pathlib import Path

import markdown
from IPython.display import Markdown, display
# from mistralai import Mistral
# from mistralai.models import OCRResponse

# Set up the API key
api_key = os.environ["MISTRAL_API_KEY"]
# client = Mistral(api_key=api_key)

In [ ]:
# # Define the file path
# file_path = r"/mnt/mydisk/Projects/plu/data/external/Grenoble/Dispositions_Generales.pdf"

# # Upload the file
# uploaded_pdf = client.files.upload(
#     file={
#         "file_name": "Dispositions_Generales.pdf",
#         "content": open(file_path, "rb"),
#     },
#     purpose="ocr",
# )

signed_url = client.files.get_signed_url(file_id="e086b2b4-82e4-45fc-be98-168361e2106e")

ocr_response = client.ocr.process(
    model="mistral-ocr-latest",
    document={
        "type": "document_url",
        "document_url": signed_url.url,
    },
    include_image_base64=True,
)

response_dict = json.loads(ocr_response.model_dump_json(indent=4))
json_string = json.dumps(response_dict, indent=4)
print(json_string)

# # Sauvegarder dans un fichier texte
# with open(
#     "/mnt/mydisk/Projects/plu/data/raw/Dispositions_Generales.json", "w", encoding="utf-8"
# ) as file:
#     file.write(json_string)

In [ ]:
def replace_images_in_markdown(markdown_str: str, images_dict: dict) -> str:
    for img_name, base64_str in images_dict.items():
        markdown_str = markdown_str.replace(
            f"![{img_name}]({img_name})", f"![{img_name}]({base64_str})"
        )
    return markdown_str


def get_combined_markdown(ocr_response: OCRResponse) -> str:
    markdowns: list[str] = []
    for page in ocr_response.pages:
        image_data = {}
        for img in page.images:
            image_data[img.id] = img.image_base64
        markdowns.append(replace_images_in_markdown(page.markdown, image_data))

    return "\n\n".join(markdowns)


display(Markdown(get_combined_markdown(ocr_response)))

In [ ]:
def save_as_html(ocr_response, output_path="ocr_result.html"):
    """Save OCR results as a single HTML file with embedded images"""
    combined_markdown = get_combined_markdown(ocr_response)

    # Convert markdown to HTML (using markdown library)
    html_content = markdown.markdown(combined_markdown)

    # Wrap in basic HTML document structure
    html_doc = f"""<!DOCTYPE html>
    <html>
    <head>
        <title>OCR Results</title>
        <meta charset="utf-8">
    </head>
    <body>
        {html_content}
    </body>
    </html>
    """

    # Write to file
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(html_doc)

    print(f"HTML saved to {output_path}")


def save_as_markdown_with_images(ocr_response, output_dir="ocr_results"):
    """Save OCR results as a markdown file with separate image files"""
    # Create directory if it doesn't exist
    Path(output_dir).mkdir(exist_ok=True)

    # Create images subdirectory
    images_dir = os.path.join(output_dir, "images")
    Path(images_dir).mkdir(exist_ok=True)

    all_markdown = []

    for i, page in enumerate(ocr_response.pages):
        page_markdown = page.markdown

        # Extract and save images
        for img in page.images:
            # Decode base64 image
            img_data = base64.b64decode(
                img.image_base64.split(",")[1]
                if "," in img.image_base64
                else img.image_base64
            )

            # Determine image format (assuming it's encoded in the base64 string)
            img_format = "png"  # Default to png
            if "image/jpeg" in img.image_base64:
                img_format = "jpg"
            elif "image/png" in img.image_base64:
                img_format = "png"

            # Save image to file
            img_filename = f"{img.id}.{img_format}"
            img_path = os.path.join(images_dir, img_filename)
            with open(img_path, "wb") as img_file:
                img_file.write(img_data)

            # Update markdown to reference local image file
            img_rel_path = os.path.join("images", img_filename).replace("\\", "/")
            page_markdown = page_markdown.replace(
                f"![{img.id}]({img.image_base64})", f"![{img.id}]({img_rel_path})"
            )

        all_markdown.append(page_markdown)

    # Write combined markdown to file
    markdown_path = os.path.join(output_dir, "ocr_results.md")
    with open(markdown_path, "w", encoding="utf-8") as md_file:
        md_file.write("\n\n".join(all_markdown))

    print(f"Markdown saved to {markdown_path}")
    print(f"Images saved to {images_dir}")


# Option 1: Save as a single HTML file
save_as_html(ocr_response, "my_ocr_results.html")

# Option 2: Save as markdown with separate image files
save_as_markdown_with_images(ocr_response, "my_ocr_results")

### Files Organisation

In [ ]:
root_path = Path(r"/mnt/mydisk/Projects/plu/data/raw/Grenoble")

folders = [x for x in root_path.iterdir() if x.is_dir()]

os.makedirs(root_path / "Zone à Urbaniser", exist_ok=True)
os.makedirs(root_path / "Zone Agricole", exist_ok=True)
os.makedirs(root_path / "Zone Dédiée", exist_ok=True)
os.makedirs(root_path / "Zone Urbaine", exist_ok=True)

for folder in folders:
    if "_AU" in folder.name:
        os.rename(folder, root_path / "Zone à Urbaniser" / folder.name)
    elif any(suffix in folder.name for suffix in ["_A", "_N"]):
        os.rename(folder, root_path / "Zone Agricole" / folder.name)
    elif any(suffix in folder.name for suffix in ["_UE", "_UV", "_UZ"]):
        os.rename(folder, root_path / "Zone Dédiée" / folder.name)
    elif any(suffix in folder.name for suffix in ["_UA", "_UB", "_UC", "_UD."]):
        os.rename(folder, root_path / "Zone Urbaine" / folder.name)

# Cleaning

## Images

In [ ]:
import os
from pathlib import Path

import imagehash
from PIL import Image

In [ ]:
def get_image_hash(image_path: Path) -> str:
    """Get perceptual hash of an image"""
    with Image.open(image_path) as img:
        img_hash = imagehash.phash(img)
    return img_hash


def reference_images(references_dir: Path) -> dict:
    """Get perceptual hash of an image"""
    img_hashes = {}
    for img_file in os.listdir(references_dir):
        img_path = os.path.join(references_dir, img_file)
        img_hash = get_image_hash(img_path)
        img_hashes[img_file] = img_hash
    return img_hashes


def is_similar_to_reference(references_dir: Path, image_path: Path, threshold: int = 5):
    """Check if an image is similar to a reference image"""
    img_hash = get_image_hash(image_path)
    reference_hashes = reference_images(references_dir=references_dir)

    for ref_hash in reference_hashes.values():
        hash_diff = img_hash - ref_hash
        if hash_diff <= threshold:
            return True

    return False


def filter_images_in_document(
    references_dir: Path, images_dir: Path, threshold: int = 5
):
    """Filter images in a directory based on similarity to reference images"""
    for filename in os.listdir(images_dir):
        if filename.endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(images_dir, filename)
            is_similar = is_similar_to_reference(
                references_dir=references_dir, image_path=img_path, threshold=threshold
            )
            if is_similar:
                os.remove(img_path)

In [ ]:
filter_images_in_document(
    r"/mnt/mydisk/Projects/plu/references/images",
    r"/mnt/mydisk/Projects/plu/data/raw/Grenoble/Par Zone Agricoles/images",
    threshold=5,
)

0

## Text

In [26]:
import multiprocessing
import os
import re

import markdown
import pandas as pd
import spacy
from IPython.display import Markdown, display
from tqdm import tqdm
from pathlib import Path

In [24]:
md_path = (
    r"/mnt/mydisk/Projects/plu/data/raw/Grenoble/Dispositions_Generales/ocr_results.md"
)
img_path = r"/mnt/mydisk/Projects/plu/data/raw/Grenoble/Dispositions_Generales/images"

with open(md_path, "r", encoding="utf-8") as file:
    md_content = file.read()

# Open images directory
img_files = os.listdir(img_path)
display(Markdown(md_content))

![img-0.jpeg](img-0.jpeg)

# RÈGLEMENT ÉCRIT 

TOME 1.1

## DISPOSITIONS GÉNÉRALES

RÈGLES COMMUNES - LEXIQUE
![img-1.jpeg](img-1.jpeg)

PLUi approuvé le 20 décembre 2019
Dernière actualisation : Modification $\mathrm{n}^{\circ} 1$ du 16 décembre 2022* * Pour plus d'information, se référer au tableau des procédures d'évolution




![img-2.jpeg](img-2.jpeg)

Plan Local d'Urbanisme Intercommunal

# Règlement pièces écrites 

## Règlement zone Règles communes

Dispositions générales - Tome 1.1
> PLUI approuvé le 20/12/2019 - Modification n¹ du 16/12/2022
![img-3.jpeg](img-3.jpeg)

CHAPITRE 1 - DESTINATION DES CONSTRUCTIONS, USAGE DES SOLS, ACTIVITES ET INSTALLATIONS, MIXITE FONCTIONNELLE ET SOCIALE ..... 5
ARTICLE 1 - CONSTRUCTIONS, USAGES ET AFFECTATIONS DES SOLS, ACTIVITÉS ET INSTALLATIONS INTERDITS ..... 5
1.1. CONSTRUCTIONS INTERDITES ..... 5
1.2. USAGES ET AFFECTATIONS DES SOLS INTERDITS ..... 6
1.3. ACTIVITÉS ET INSTALLATIONS INTERDITES ..... 7
ARTICLE 2 - CONSTRUCTIONS, USAGES ET AFFECTATIONS DES SOLS, ACTIVITÉS ET INSTALLATIONS SOUMISES À CONDITIONS PARTICULIÈRES ..... 7
2.1. CONSTRUCTIONS SOUMISES À DES CONDITIONS PARTICULIÈRES ..... 7
2.2. USAGES ET AFFECTATIONS DES SOLS SOUMIS À DES CONDITIONS PARTICULIÈRES ..... 9
2.3. ACTIVITÉS ET INSTALLATIONS SOUMISES À DES CONDITIONS PARTICULIÈRES ..... 10
ARTICLE 3 - MIXITÉ FONCTIONNELLE ET SOCIALE ..... 11
3.1. DISPOSITIONS EN FAVEUR DE LA MIXITÉ COMMERCIALE ET FONCTIONNELLE ..... 11
3.2. RÈGLES DIFFÉRENCIÉES ENTRE REZ-DE-CHAUSSÉE ET ÉTAGES SUPÉRIEURS ..... 11
3.3. DISPOSITIONS EN FAVEUR DE LA MIXITÉ SOCIALE ..... 11
CHAPITRE 2 - CARACTERISTIQUES URBAINES, ARCHITECTURALES, ENVIRONNEMENTALES ET PAYSAGERES ..... 15
ARTICLE 4 - IMPLANTATION ET VOLUMÉTRIE DES CONSTRUCTIONS ET DES INSTALLATIONS ..... 15
4.1. IMPLANTATION PAR RAPPORT AUX VOIES ET EMPRISES PUBLIQUES ..... 15
4.2. IMPLANTATION DES CONSTRUCTIONS PAR RAPPORT AUX LIMITES SÉPARATIVES ..... 18
4.3. IMPLANTATION DES CONSTRUCTIONS LES UNES PAR RAPPORT AUX AUTRES SUR UNE MÊME PROPRIÉTÉ ..... 19
4.4. EMPRISE AU SOL DES CONSTRUCTIONS ..... 19
4.5. COEFFICIENT D'EMPRISE AU SOL MINIMUM ET HAUTEUR MINIMUM AU SEIN DES PÉRIMÈTRES D'INTENSIFICATION URBAINE ..... 20
4.6. HAUTEUR DES CONSTRUCTIONS ET DES INSTALLATIONS ..... 21
ARTICLE 5 - QUALITÉ URBAINE, ARCHITECTURALE, ENVIRONNEMENTALE ET PAYSAGÈRE ..... 24
5.1. INSERTION DES CONSTRUCTIONS ET DES INSTALLATIONS DANS LEUR ENVIRONNEMENT ..... 24
5.2. CARACTÉRISTIQUES ARCHITECTURALES DES FAÇADES ET TOITURES ..... 24
5.3. CARACTÉRISTIQUES DES CLÔTURES ..... 25
5.4. PRESCRIPTIONS RELATIVES AU PATRIMOINE BÂTI ET PAYSAGER À PROTÉGER, À CONSERVER, À RESTAURER, À METTRE EN VALEUR OU À REQUALIFIER ..... 27
ARTICLE 6 - TRAITEMENT ENVIRONNEMENTAL ET PAYSAGER DES ESPACES NON BÂTIS, DES CONSTRUCTIONS ET DE LEURS ABORDS ..... 28

6.1. OBLIGATIONS EN MATIÈRE DE RÉALISATION D'ESPACES LIBRES ET DE PLANTATIONS, D'AIRES DE JEUX ET DE LOISIRS ..... 28
6.2. SURFACES VÉGÉTALISÉES OU PERMÉABLES ..... 28
6.3. MAINTIEN OU REMISE EN ÉTAT DES CONTINUITÉS ÉCOLOGIQUES ..... 32
6.4. GESTION DES EAUX PLUVIALES ET DU RUISSELLEMENT ..... 33
6.5. AMÉNAGEMENT D'EMPLACEMENTS SPÉCIFIQUES DÉDIÉS À LA COLLECTE DES DÉCHETS MÉNAGERS ET ASSIMILÉS ..... 33
CHAPITRE 3 - EQUIPEMENTS ET RESEAUX ..... 34
ARTICLE 7 - STATIONNEMENT ..... 34
7.1. STATIONNEMENT DES VÉHICULES MOTORISÉS ..... 34
7.2. STATIONNEMENT DES CYCLES ..... 40
ARTICLE 8 - DESSERTE PAR LES VOIES PUBLIQUES ET PRIVÉES ..... 45
8.1. ACCÈS ..... 45
8.2. VOIRIES ..... 45
ARTICLE 9 - DESSERTE PAR LES RÉSEAUX ..... 46
9.1. ALIMENTATION EN EAU POTABLE ..... 46
9.2. GESTION DES EAUX USÉES DOMESTIQUES ..... 46
9.3. GESTION DES EAUX USÉES NON DOMESTIQUES ..... 46
9.4. UTILISATION DU RÉSEAU D'EAUX PLUVIALES ..... 47
9.5. RÉSEAUX ÉLECTRIQUES ET TÉLÉPHONIQUES ..... 47
9.6. DÉPLOIEMENT DE LA FIBRE OPTIQUE ..... 47
ARTICLE 10 - ENERGIE ET PERFORMANCES ÉNERGÉTIQUES ..... 48

# PREAMBULE 

La vocation des zones figure dans le rapport de présentation (Tome 4 Livret métropolitain).

Le règlement de chaque zone est complété par les Dispositions générales, qui comprennent :

- les règles communes (en complément ou à défaut de règles particulières dans les zones),
- le lexique, qui comprend la définition d'un certain nombre de mots et de notions utilisés dans le texte du règlement,
- le règlement des risques,
- le règlement du patrimoine.

Les règles communes et les règlements de zone s'appliquent sauf dispositions contraires des règlements du patrimoine ou des risques.

Les projets doivent être compatibles avec les orientations d'aménagement et de programmation (OAP) thématiques Paysage et biodiversité, Qualité de l'Air, et Risques et résilience et le cas échéant dans les secteurs de projet avec les OAP sectorielles, qui apportent toutes des compléments pour l'insertion des projets dans leur environnement.

Les constructions établies préalablement à l'approbation du PLUi (le 20/12/2019) et ne respectant pas les règles du règlement du PLUi peuvent faire l'objet de transformations, d'extensions ou de changements de destination, à condition que les travaux rendent la construction existante plus conforme aux dispositions réglementaires ou bien qu'ils soient sans effet vis-à-vis de ces dispositions.

Lorsque le projet concerne un terrain comprenant plusieurs parcelles, les règles s'appliquent à l'unité foncière.

# CHAPITRE 1 - DESTINATION DES CONSTRUCTIONS, USAGE DES SOLS, ACTIVITES ET INSTALLATIONS, MIXITE FONCTIONNELLE ET SOCIALE 

Le chapitre 1 énumère ce qui est autorisé et interdit dans la zone

- L'article 1 énumère ce qui est interdit.
- L'article 2 ce qui est autorisé sous condition.

Dans les zones U, tout ce qui n'est pas mentionné par les articles 1 et 2 est autorisé (sans condition).
Dans les zones AU (strict) A, AL, N et NL, tout ce qui n'est pas autorisé par l'article 2 est interdit.
Les destinations et sous-destinations de constructions sont listées aux articles 1.1 (interdites) et 2.1 (autorisées sous condition). Elles sont définies par le code de l'urbanisme et sont détaillées dans l'arrêté du 10 novembre 2016, reproduit en fin de lexique.

Les usages et affectations des sols figurent aux articles 1.2 (interdits) et 2.2 (autorisés sous condition).
Les activités et les installations sont énumérées aux articles1.3 (interdites) et 2.3 (autorisées sous condition).

## ARTICLE 1 - CONSTRUCTIONS, USAGES ET AFFECTATIONS DES SOLS, ACTIVITÉS ET INSTALLATIONS INTERDITS

### 1.1. Constructions interdites

En complément des règles figurant ci-dessous, se reporter aux dispositions de l'article 1.1 des règlements de zone et aux déclarations d'utilité publique relatives aux captages (DUP, dans l'annexe 2A2 du PLUi).

Dans les périmètres de protection des captages, identifiés sur le document graphique B3 «Plan de prévention des pollutions», nonobstant toute disposition contraire des règlements de zones et des déclarations d'utilité publique relatives aux captages (DUP, dans l'annexe 2A2 du PLUi), sont interdits :

## - Dans les périmètres immédiats :

Toutes les constructions à l'exception des locaux techniques et industriels des administrations publiques et assimilés, (qui sont autorisés sous condition, cf art 2.1).

- Dans les périmètres rapprochés, sauf en périmètres rapprochés $B$ et $C$, les constructions destinées :
. aux exploitations agricoles et forestières ;
. au logement (y compris annexes et extensions) et à l'hébergement ;
. au commerce et aux activités de services ;
. aux autres activités des secteurs secondaires et tertiaires.
. aux équipements d'intérêt collectif et aux services publics, à l'exception des sous-destinations suivantes qui sont autorisés sous condition, (cf art 2.1) :
- les locaux techniques et industriels des administrations publiques et assimilés,
- les établissements d'enseignement, de santé et d'action sociale.

# - Dans les périmètres rapprochés de type C, les constructions destinées : 

à l'hébergement ;
au commerce et aux activités de services;
aux autres activités des secteurs secondaires et tertiaires.
aux équipements d'intérêt collectif et aux services publics, à l'exception des sous destinations suivantes qui sont autorisés sous condition, (cf art 2.1) :

- les locaux techniques et industriels des administrations publiques et assimilés,
- les établissements d'enseignement, de santé et d'action sociale.
- Dans les périmètres éloignés de type 1, les constructions destinées à l'industrie.
- Dans les périmètres éloignés de type 1 et 2, les centres de congrès et d'exposition.


### 1.2. Usages et affectations des sols interdits

En complément des règles figurant ci-dessous, se reporter aux dispositions de l'article 1.2 des règlements de zone et aux déclarations d'utilité publique relatives aux captages (DUP, dans l'annexe 2A2 du PLUi).

Dans l'ensemble des périmètres de protection des captages, identifiés sur le document graphique B3 «Plan de prévention des pollutions», nonobstant toute disposition contraire des règlements de zones, sont interdits :

- Les parcs résidentiels de loisirs
- Les parcs d'attraction
- L'aménagement de terrains pour la pratique de sports ou de loisirs motorisés
- L'aménagement de terrains pour la pratique du golf
- Les garages collectifs de caravanes ou de résidences mobiles de loisirs
- Les dépôts de véhicules
- Les aires d'accueil et les terrains familiaux des gens du voyage
- Les résidences démontables constituant l'habitat permanent de leurs utilisateurs
- Les habitations légères de loisirs (HLL), sauf en secteur ALt2
- La pose de canalisations de transport de matières dangereuses.


## - Dans les périmètres immédiats :

- Les affouillements et exhaussements du sol, sauf pour les puits et ouvrages nécessaires à l'exploitation des captages.
- Les forages dans la nappe, les prélèvements d'eau dans les milieux superficiels, sauf pour les puits et ouvrages nécessaires à l'exploitation des captages.

# - Dans les périmètres rapprochés: 

- Les forages dans la nappe, les prélèvements d'eau dans les milieux superficiels.
- Les affouillements et exhaussements du sol nécessaires à la création de retenues d'eau.


## - Dans les périmètres immédiats et rapprochés :

- L'aménagement de terrains pour la pratique du camping
- Les aires de jeu et de sport
- Les piscines.


## - Dans l'ensemble des périmètres sauf dans les périmètres éloignés de type 2 :

- Les dépôts en plein air de matériaux ou de déchets.


### 1.3. Activités et installations interdites

En complément des règles figurant ci-dessous, se reporter aux dispositions de l'article 1.3 des règlements de zone et aux déclarations d'utilité publique relatives aux captages (DUP, dans l'annexe 2A2 du PLUi).

- Les ouvrages et accessoires des lignes de distribution d'énergie, de production d'électricité ou de transport par câble, tels que pylônes, antennes relai, éoliennes... et les antennes d'émission ou de réception (radios, télévisions, radiotéléphones), sont interdits :
. sur les terrains et bâtiments protégés au titre du patrimoine, en niveaux 2 et 3, repérés sur le document graphique F2 «Plan du patrimoine bâti, paysager et écologique».
. dans les périmètres de protection immédiats et rapprochés des captages.
- Les carrières sont interdites dans l'ensemble des périmètres de protection des captages identifiés sur le document graphique B3 «Plan de prévention des pollutions», nonobstant toute disposition contraire des règlements de zones.


## ARTICLE 2 - CONSTRUCTIONS, USAGES ET AFFECTATIONS DES SOLS, ACTIVITÉS ET INSTALLATIONS SOUMISES À CONDITIONS PARTICULIÈRES

### 2.1. Constructions soumises à des conditions particulières

En complément des règles figurant ci-dessous, se reporter aux dispositions de l'article 2.1 des règlements de zone.

Dispositions particulières relatives à l'application des PAPA (périmètres d'attente de projets d'aménagement)
Lorsque l'unité foncière où se situe le projet est couverte par un PAPA (périmètre d'attente d'un projet d'aménagement), délimité sur le document graphique $G$ «Plan des OAP et secteurs de projet », les constructions, installations ou extensions créant plus de $40 \mathrm{~m}^{2}$ de surface de plancher sont interdites pendant une durée maximum de 5 ans à compter de la création du PAPA.

| PAPA - Commune - nom du périmètre | Procédure PLUi ayant créé le PAPA | Date de fin d'opposabilité du PAPA <br> Les PAPA sont valables pendant 5 ans maximum à compter du caractère exécutoire de la procédure PLUI les ayant mis en place. |
| :--: | :--: | :--: |
| PAPA 1 - BRIE -ET-ANGONNES - Tavernolles PAPA 2 - MEYLAN - Pharma <br> PAPA 3 - GRENOBLE - Alma/Maréchal Leclerc PAPA 4 - GRENOBLE - Malraux/Hoche/Lyautey PAPA 5 - VUDLLE - Frühe Cros/Jean Jaurès PAPA 7 - JAERIE - Route du Plâtre PAPA 8 - SAINT-EGREVE - Piscine des Mails | Approbation du PLUI le 20/12/2019 exécutoire le 28/01/2020 | $27 / 01 / 2025$ |
| PAPA 10 - MEYLAN - Avenue du Granier | Modification n'1 du PLUi approuvée le 16/12/2022 Exécutoire le XX/XX/XX | XX/XX/XX |

Dans les périmètres de protection des captages, identifiés sur le document graphique B3 «Plan de prévention des pollutions», les constructions listées ci-dessous sont autorisées sous réserve :

- qu'elles ne soient pas interdites par les déclarations d'utilité publique relatives aux captages (DUP, dans l'annexe 2A2 du PLUi) ou les règlements de zones ;
- qu'elles n'aggravent pas la vulnérabilité des captages ;
- d'une gestion appropriée des eaux usées et des eaux pluviales, conformément aux dispositions des articles 9.2 à 9.4 du présent règlement ;
- d'une limitation au strict nécessaire des affouillements et exhaussements de sol pour les constructions autorisées. Les affouillements ne devront pas modifier les écoulements souterrains et superficiels. Les matériaux utilisés en cas d'exhaussement devront être vierges et non pollués.

Ces conditions sont cumulatives :

# - Dans l'ensemble des périmètres : 

les reconstructions partielles ou totales après sinistre.

## - Dans les périmètres immédiats :

les constructions destinées aux locaux techniques et industriels des administrations publiques et assimilés, s'ils sont nécessaires au fonctionnement des captages.

- Dans les périmètres rapprochés, sauf en périmètres rapprochés $B$ et $C$, les constructions destinées :
aux locaux techniques et industriels des administrations publiques et assimilés, à condition qu'ils n'entravent pas le bon fonctionnement des captages ;
aux extensions des établissements d'enseignement, de santé et d'action sociale.
- Dans les périmètres rapprochés de type B et C, les constructions destinées :
aux exploitations agricoles et forestières, sous réserve qu'elles soient autorisées par la déclaration d'utilité publique relative aux captages (DUP, dans l'annexe 2A2 du PLUi) ;
aux logements (y compris annexes et extensions), sous réserve qu'ils soient raccordés à l'assainissement collectif ;

. aux locaux techniques et industriels des administrations publiques et assimilés, à condition qu'ils n'entravent pas le bon fonctionnement des captages ;
. aux extensions des établissements d'enseignement, de santé et d'action sociale.

- Dans les périmètres éloignés de type 1 et 2, les constructions destinées :
. aux exploitations agricoles et forestières ;
à l'habitation (logement et hébergement)
à l'artisanat et au commerce de détail.
. aux autres sous-destinations de la destination «commerce et des activités de service» (restauration, commerce de gros, activités de services avec accueil de clientèle, cinéma, hôtels et autres hébergements touristiques), sous réserve que les constructions n'entrainent aucune artificialisation supplémentaire de terrain ;
. aux équipements d'intérêt collectif et services publics;
. aux entrepôts, et bureaux
- Dans les périmètres éloignés de type 2 :

Les constructions destinées à l'industrie

# 2.2. Usages et affectations des sols soumis à des conditions particulières 

En complément des règles figurant ci-dessous, se reporter aux dispositions de l'article 2.2 des règlements de zone.
Dans les périmètres de protection des captages, identifiés sur le document graphique B3 «Plan de prévention des pollutions», sont autorisés sous réserve :

- qu'ils ne soient pas interdits par les déclarations d'utilité publique relatives aux captages (DUP, dans l'annexe 2A2 du PLUi) ou par les règlements de zones ;
- qu'ils n'aggravent pas la vulnérabilité des captages ;
- d'une gestion appropriée des eaux usées et des eaux pluviales, conformément aux dispositions des articles 9.2 à 9.4 du présent règlement ;
- d'une limitation au strict nécessaire des affouillements et exhaussements de sol pour les constructions autorisées. Les affouillements ne devront pas modifier les écoulements souterrains et superficiels. Les matériaux utilisés en cas d'exhaussement devront être vierges et non pollués.

Ces conditions sont cumulatives :

## - Dans l'ensemble des périmètres :

- Les travaux liés à l'amélioration de la protection des canalisations existantes.


## - Dans les périmètres immédiats :

- Les exhaussements et affouillements à condition qu'ils soient liés aux puits et ouvrages nécessaires à l'exploitation des captages.

# - Dans les périmètres rapprochés et éloignés de type 1 et 2 : 

- Les exhaussements et affouillements du sol, à condition qu'ils soient liés aux puits et ouvrages nécessaires à l'exploitation des captages ou aux constructions autorisées.


## - Dans les périmètres éloignés de type 1 et 2 :

- Les forages dans la nappe, les prélèvements d'eau dans les milieux superficiels ;
- L'aménagement de terrains pour la pratique du camping
- Les aires de jeu et de sport
- Les piscines
- Les exhaussements et affouillements du sol nécessaires à la création de retenues d'eau.


## Dans les périmètres éloignés de type 2 :

- Les dépôts en plein air de matériaux ou de déchets.


### 2.3. Activités et installations soumises à des conditions particulières

En complément des règles figurant ci-dessous, se reporter aux dispositions de l'article 2.3 des règlements de zone.
Dans les périmètres de protection éloignés de type 1 et 2 des captages identifiés sur le document graphique B3 « Plan de prévention des pollutions » :
les ouvrages et accessoires des lignes de distribution d'énergie, de production d'électricité ou de transport par câble, tels que pylônes, antennes relai, éoliennes... et les antennes d'émission ou de réception (radios, télévisions, radiotéléphones), sont autorisés sous réserve :

- qu'ils ne soient pas interdits par les déclarations d'utilité publique relatives aux captages (DUP, dans l'annexe 2A2 du PLUi) ou les règlements de zones ;
- qu'ils n'aggravent pas la vulnérabilité des captages ;
- d'une gestion appropriée des eaux usées et des eaux pluviales, conformément aux dispositions des articles 9.2 à 9.4 du présent règlement ;
- d'une limitation au strict nécessaire des affouillements et exhaussements de sol pour les constructions autorisées. Les affouillements ne devront pas modifier les écoulements souterrains et superficiels. Les matériaux utilisés en cas d'exhaussement devront être vierges et non pollués.

Ces conditions sont cumulatives.

# ARTICLE 3 - MIXITÉ FONCTIONNELLE ET SOCIALE 

### 3.1. Dispositions en faveur de la mixité commerciale et fonctionnelle

## - Dispositions applicables le long des linéaires de mixité commerciale

Dans les nouvelles constructions, lors de changements de destination ou lors de la transformation des constructions existantes, les rez-de-chaussée côté rue (hors hall d'entrée, surfaces destinées au stationnement des vélos, accès au stationnement des véhicules légers et locaux techniques) doivent être uniquement dédiés :

- le long des linéaires de préservation de l'artisanat et du commerce (L1) repérés sur le document graphique C1 «Atlas de la mixité fonctionnelle et commerciale » : à l'artisanat et au commerce de détail ou à la restauration ;
- le long des linéaires de préservation de l'artisanat, du commerce et des services (L2) repérés sur le document graphique C1 « Atlas de la mixité fonctionnelle et commerciale »: à l'artisanat et au commerce de détail, à la restauration, aux activités de services où s'effectue l'accueil d'une clientèle ou aux équipements d'intérêt collectif et services publics.


## - Dispositions applicables le long des linéaires de mixité fonctionnelle

Le long des linéaires de mixité fonctionnelle (L3) repérés sur le document graphique C1 « Atlas de la mixité fonctionnelle et commerciale »: Dans les nouvelles constructions, lors de changements de destination ou lors de la transformation des constructions existantes, sont interdits en rez-de-chaussée côté rue les logements et le stationnement (hors hall d'entrée, surfaces destinées au stationnement des vélos, accès au stationnement des véhicules légers et locaux techniques).

### 3.2. Règles différenciées entre rez-de-chaussée et étages supérieurs

## - Hauteur sous dalle

Les rez-de-chaussée destinés à accueillir l'artisanat et le commerce de détail, la restauration et les activités de services où s'effectue l'accueil d'une clientèle, doivent avoir une hauteur sous dalle de $3,5 \mathrm{~m}$ minimum ou au moins égale à celle des constructions adjacentes, dans le cas où celles-ci auraient des hauteurs de rez-de-chaussée plus importantes, de façon à harmoniser les hauteurs de rez-de-chaussée sur rue.

### 3.3. Dispositions en faveur de la mixité sociale

Le "logement" et l'"hébergement" constituent les sous-destinations de la destination "habitation", et sont définies par le code de l'urbanisme (se reporter en fin de lexique pour information).

Sont comptabilisés dans le pourcentage de logements sociaux :

- Les logements et hébergements locatifs sociaux financés par des prêts type :
- PLAl : prêt locatif aidé,
- PLUS : prêt locatif à usage social,
- PLS : prêt locatif social
- Les logements en accession sociale via le Bail Réel Solidaire (BRS)

Dans les secteurs de mixité sociale des communes déficitaires, une part minimale globale de PLUS et PLAI est systématiquement exigée, ainsi qu'une part minimale de PLAI.

Selon le contexte des communes au regard de leurs obligations fixées par la loi SRU modifiée, des règles particulières s'appliquent comme indiqué sur l'atlas de la mixité sociale (C2).

En complément de cet atlas, il convient de se référer à «l'explication de la légende du plan» (cf infra) pour les secteurs de mixité sociale et à la liste T6.2 pour les emplacements réservés de mixité sociale.

- DANS LES SECTEURS DE MIXITE SOCIALE délimités sur le document graphique C2 «Atlas de la mixité sociale»:


# Seuil de déclenchement 

Doivent répondre aux prescriptions mentionnées sur ce plan, les opérations :

- comprenant de l'habitation, qu'il s'agisse d'opérations nouvelles de construction, d'extensions, de réhabilitations ou de changements de destination,
- et créant au minimum, en nombre ou en surface, selon les communes :
- au moins 3 habitations ou $210 \mathrm{~m}^{2}$ de surface de plancher d'habitation (logement / hébergement)
- au moins 5 habitations ou $350 \mathrm{~m}^{2}$ de surface de plancher d'habitation (logement / hébergement)
- au moins 10 habitations ou $700 \mathrm{~m}^{2}$ de surface de plancher d'habitation (logement / hébergement)
- au moins 20 habitations ou $1400 \mathrm{~m}^{2}$ de surface de plancher d'habitation (logement / hébergement)
- au moins 25 habitations ou $1750 \mathrm{~m}^{2}$ de surface de plancher d'habitation (logement / hébergement)

La réalisation d'une seule habitation ou de l'extension d'une seule habitation n'est pas soumise à ces prescriptions.
Les taux de mixité sociale indiqués sur le document graphique C2 «Atlas de la mixité sociale» s'appliquent :

- au nombre d'habitations (logements ou d'hébergements) et à la surface de plancher.
- à chaque permis de construire ou permis d'aménager déposé sur une unité foncière ou sur plusieurs unités foncières contigües.

En cas de création de logements ou d'hébergements financés en PLAI, les taux s'appliquent au nombre d'habitations ou à la surface de plancher.

En cas d'opération mixte (habitation + autre destination), ils ne s'appliquent qu'à la part de surface de plancher consacrée à l'habitation.

Le nombre de logements ou d'hébergements à réaliser et leur surface constituent des minima (ou des maximas pour les PLS et les BRS) ; leur nombre est arrondi à l'entier le plus proche ; l'entier et demi est arrondi à l'entier supérieur.

Dans le cadre de la réalisation d'une opération d'aménagement d'ensemble, l'obligation définie par le plan peut être programmée sur l'ensemble du périmètre de l'opération, à condition que la répartition des logements / hébergements locatifs sociaux au sein dudit périmètre permette de respecter l'obligation globale.

# Explication de la légende du plan pour les secteurs de mixité sociale (SMS) 

Les taux de mixité sociale sont indiqués sur le document graphique C2 «Atlas de la mixité sociale » à travers des libellés de trois types, explicités dans la légende de l'atlas C2 et dans le tableau ci-après. Ils précisent :

- Le type de secteur de mixité sociale
- Le seuil de déclenchement de la règle : s'applique au nombre d'habitations (logements / hébergements) ou à la surface de plancher
- Les taux (minimum ou maximum) imposés selon le type d'habitations à réaliser : s'appliquent au nombre d'habitations (logements / hébergements) et à la surface de plancher (ou à la surface de plancher pour les PLAi).
Les trois libellés du tableau suivant sont des exemples de combinaisons:

| Libellé des SMS inscrits à l'atlas C2 de la mixité sociale | Type de secteur de mixité sociale | Seuil de déclenchement de la règle en nombre d'habitations ou en surface de plancher | Taux minimal imposé de logements/ hébergements locatifs sociaux : PLUS et PLAL à l'échelle de l'opération | Taux minimal de PLAi à réaliser dans le part de logements / hébergements locatifs sociaux imposés | Taux minimal imposé de logements / hébergements en accession à l'échelle de l'opération |
| :--: | :--: | :--: | :--: | :--: | :--: |
| LS10.20.30 | $\begin{aligned} & \text { LS10.20.30 : } \\ & \text { Secteur de } \\ & \text { mixité sociale } \\ & \text { relatif à } \\ & \text { l'intégration de } \\ & \text { logements / } \\ & \text { hébergements } \\ & \text { locatifs sociaux } \end{aligned}$ | LS10.20.30 : <br> à partir de 10 habitations ou $700 \mathrm{~m}^{2}$ de surface de plancher | $\begin{aligned} & \text { LS10.20.30 : } \\ & \text { au moins } 20 \% \text { de } \\ & \text { logements / } \\ & \text { hébergements } \\ & \text { locatifs sociaux } \\ & \text { PLUS PLAi } \end{aligned}$ | $\begin{aligned} & \text { LS10.20.30 : } \\ & \text { dont au moins } \\ & 30 \% \text { de PLAi } \end{aligned}$ | non concerné par ce secteur de mixité |
| LA3.90 | $\begin{aligned} & \text { LA3.90 : } \\ & \text { Secteur de } \\ & \text { mixité sociale } \\ & \text { relatif à } \\ & \text { l'intégration de } \\ & \text { logements / } \\ & \text { hébergements } \\ & \text { en accession } \end{aligned}$ | LA3. 90 : <br> à partir de 3 habitations ou $210 \mathrm{~m}^{2}$ de surface de plancher | non concerné par ce secteur de mixité | non concerné par ce secteur de mixité | $\begin{aligned} & \text { LA3.90 : } \\ & \text { au moins } 90 \% \text { de } \\ & \text { logements / } \\ & \text { hébergements en } \\ & \text { acceccion } \end{aligned}$ |
| SA20.25.70 | SA20.25.70 : <br> Secteur de mixité sociale relatif à l'intégration de logements / hébergements locatifs sociaux et en accession | SA20.25.70 : <br> à partir de 20 habitations ou $1400 \mathrm{~m}^{2}$ de surface de plancher | SA20.25.70 : <br> au moins $25 \%$ de logements / hébergements locatifs sociaux PLUS PLAi | non concerné par ce secteur de mixité | SA20.25.70 : <br> au moins $70 \%$ de logements / hébergements en acceccion |
|  |  |  |  |  | Taux maximal de <br> PLS et ou BRS |
| S3.50.40.40.10 | $\begin{aligned} & \text { S3.50.40.40.105 } \\ & \text { ecteur de mixité } \\ & \text { sociale relatif à } \\ & \text { l'intégration de } \\ & \text { logements / } \\ & \text { hébergements } \\ & \text { sociaux avec } \\ & \text { taux de PLS } \\ & \text { et/ou BRS } \\ & \text { plafonné } \end{aligned}$ | SA.50.40.40.10 <br> à partir de 3 habitations ou $210 \mathrm{~m}^{2}$ de surface de plancher | $\begin{aligned} & \text { SA.50.40.40.10 } \\ & \text { au moins } 50 \% \text { de } \\ & \text { logements / } \\ & \text { hébergements } \\ & \text { sociaux PLUS - } \\ & \text { PLAi - PLS - BRS } \end{aligned}$ | $\begin{aligned} & \text { S3.50.40.40.10 } \\ & \text { Taux minimal de } \\ & \text { PLUS - PLAi } \\ & \hline \text { SA.50.40.40.10 } \\ & \text { Taux minimal de } \\ & \text { PLAi } \end{aligned}$ | Taux maximal de <br> PLS et ou BRS pour le programme total de l'opération <br> S3.50.40.40.10 |

# - DANS LES EMPLACEMENTS RESERVES POUR L'HABITAT (ERS) : 

Les constructions sont autorisées dans la mesure où elles respectent le programme de logements / d'hébergements indiqué sur le document graphique C2 «Atlas de la mixité sociale» et dans le tome 6_2 du règlement écrit «liste des ER mixité sociale».

Le nombre de logements / d'hébergements à réaliser constitue un minimum ; il est arrondi à l'entier le plus proche ; l'entier et demi est arrondi à l'entier supérieur.

Les pourcentages de PLAI à réaliser s'appliquent au nombre de logements / d'hébergements ou à la surface de plancher.

Dans le cadre de la réalisation d'une opération d'aménagement d'ensemble, l'obligation définie par l'ERS peut être programmée sur l'ensemble du périmètre de l'opération, à condition que la répartition des logements / hébergements locatifs sociaux au sein dudit périmètre permette de respecter l'obligation globale.

A défaut, un permis de construire ou d'aménager déposé sur une partie de l'emplacement réservé doit comporter la totalité du programme prévu par l'ERS.

| Explication de la légende du plan et de la liste des emplacements réservés pour la mixité sociale (ERS) |  |  |  |
| :--: | :--: | :--: | :--: |
| Légende | 3 premières lettres | chiffre | trigramme (3 lettres) |
| ERS_2_COR | ERS = emplacement réservé pour la mixité sociale | Numéro d'ERS | Nom de la commune |

# CHAPITRE 2 - CARACTERISTIQUES URBAINES, ARCHITECTURALES, ENVIRONNEMENTALES ET PAYSAGERES 

## ARTICLE 4 - IMPLANTATION ET VOLUMÉTRIE DES CONSTRUCTIONS ET DES INSTALLATIONS

Dans le périmètre du site patrimonial remarquable (SPR), comprenant notamment le centre ancien de Grenoble et la Bastille, il convient de s'assurer que le projet est conforme aux règles du site patrimonial remarquable (cf. Annexes, servitudes d'utilité publique, document 1C SPR de Grenoble).

## - Règles générales d'implantation par rapport aux voies, emprises publiques et limites séparatives

Les règles d'implantation par rapport aux voies, emprises publiques et limites séparatives s'appliquent aux constructions principales et aux annexes, sauf si le règlement des zones en dispose autrement.

Les règles d'implantation par rapport aux voies, emprises publiques et limites séparatives ne s'appliquent pas :

- aux ouvrages en sous-sol, aux rampes d'accès, aux terrasses de plain-pied, aux cuves ou réservoirs ;
- en cas de réhabilitation d'un bâtiment existant à la date d'approbation du PLUi (le 20/12/2019), dans la limite du volume existant.

En cas de mise en œuvre d'une isolation en saillie des façades sur une construction existante à la date d'approbation du PLUI (le 20/12/2019), nonobstant les règles d'implantation du PLUI par rapport aux voies, emprises publiques et limites séparatives, un dépassement de 30 centimètres par rapport au nu extérieur de la façade avant travaux est autorisé.

Dans le périmètre d'un plan masse, délimité sur le document graphique G2 «Atlas des secteurs de plan masse», les règles d'implantation et les règles applicables sont celles définies par le plan masse.

Les piscines (margelles et plages comprises) ainsi que les constructions accessoires et installations techniques qui leur sont associées, doivent être implantées à une distance minimale de 3 m de l'alignement ou de la limite de fait et des limites séparatives.

Une implantation autre que celle prévue par la règle générale ou par le document graphique D1 «Atlas des formes urbaines : implantations et emprises » peut être admise ou imposée pour assurer la conformité de la construction avec les prescriptions réglementaires d'un plan de prévention des risques ou prendre en compte les cartes d'aléas ou pour réduire l'exposition du projet par rapport aux risques naturels.

### 4.1. Implantation par rapport aux voies et emprises publiques

En complément des règles figurant ci-dessous, il convient de se reporter aux dispositions de l'article 4.1 des règlements de zone.

Si l'accès résulte d'une servitude de passage et non pas d'une voie de desserte, il sera fait application de l'article 4.2 pour l'implantation des constructions.

L'implantation des installations par rapport aux voies et emprises publiques n'est pas règlementée.

## - Saillies

Les saillies formées par les éléments fixes des bâtiments tels que les balcons, les volumes habitables, les encorbellements, les éléments de décor architecturaux, les débords de toitures, les escaliers extérieurs non fermés, les

pare-soleil, les auvents et marquises ou tout autre élément sur les façades vitrées permettant la protection solaire des constructions, les dispositifs nécessaires à l'utilisation des énergies renouvelables, sont autorisés à condition que la largeur de la voie (de façade à façade) soit supérieure ou égale à 8 m :

- avec un dépassement maximum de l'alignement par rapport au nu de la façade des bâtiments et une hauteur par rapport aux espaces circulables conformes au schéma ci-dessous.

Toutefois, lorsque ces saillies représentent plus de 50\% de la façade, elles doivent servir de référence pour définir la position de la façade par rapport à l'alignement ou à la limite de fait ;

Lorsque la largeur du trottoir situé au droit du projet ne figure pas sur les schémas, il faut se référer au schéma présentant la largeur de trottoir immédiatement supérieure. Lorsque le trottoir situé au droit du projet a une largeur supérieure à 4 m , c'est le schéma présentant un trottoir de 4 m qui s'applique.

En l'absence de trottoir, il faut se référer au schéma présentant un trottoir d'1 mètre et la saillie doit être implantée à 5 m de hauteur minimum.
![img-4.jpeg](img-4.jpeg)

![img-5.jpeg](img-5.jpeg)
(Schémas à valeur règlementaire)

Sont admis à l'intérieur des marges de recul prévues par l'article 4 des règlements de zone ou par le document graphique D1 «Atlas des formes urbaines: implantations et emprises», sous réserve du respect des conditions d'insertion urbaine et de traitement paysager prévues notamment aux articles 5 et 6 des règlements de zone et des règles communes :

- Les saillies formées par les éléments fixes des bâtiments tels que les balcons, les volumes habitables, les encorbellements, les éléments de décor architecturaux, les débords de toitures, les escaliers extérieurs non fermés, les pare-soleil, les auvents et marquises ou tout autre élément sur les façades vitrées permettant la protection solaire des constructions, les dispositifs nécessaires à l'utilisation des énergies renouvelables, avec un dépassement maximum de 1 mètre par rapport au nu de la façade des bâtiments et à condition d'être situés à une hauteur d'au moins 5 m par rapport aux espaces circulables.

Toutefois lorsque ces saillies représentent plus de $50 \%$ de la façade, elles doivent servir de référence pour définir la position de la façade par rapport à la marge de recul ;

- Les ouvrages en sous-sols, dans la limite d'une émergence de 1 mètre de hauteur par rapport au terrain naturel ;
- L'isolation par l'extérieur de la façade lorsqu'un bâtiment existant fait l'objet de transformation visant à améliorer ses performances énergétiques, phoniques et thermiques ;
- Les éléments permettant d'améliorer l'accessibilité des personnes à mobilité réduite aux bâtiments et installations de toute nature ;
- Les aires et locaux dédiés au stationnement des vélos ou à la collecte des déchets ménagers.

- Atlas des formes urbaines : "implantations et emprises" (document graphique D1) et "hauteurs" (document graphique D2)

Les prescriptions figurant sur I'Atlas des "formes urbaines : implantations et emprises"(document graphique D1) ainsi que sur I'Atlas des "formes urbaines : hauteurs" (document graphique D2) s'appliquent aux constructions (et non aux installations) et prévalent sur les règles d'implantation par rapport aux voies, figurant dans les règlements de zone. Ainsi, lorsque le document graphique D1 et/ou D2 «Atlas des formes urbaines » contient des prescriptions concernant l'implantation ou la hauteur maximale d'une construction (majorée ou minorée), ces prescriptions s'imposent par rapport aux règles générales d'implantation par rapport aux voies et emprises publiques figurant dans les règlements de zone, qui de ce fait ne sont pas à prendre en compte.

Lorsqu'au document graphique D1 «Atlas des formes urbaines : implantations et emprises »:

- une ligne d'implantation est inscrite, le corps principal de la construction doit être implanté sur cette ligne d'implantation,
- un trait continu ( $\qquad$ ) (alignement continu) est inscrit, les constructions doivent être implantées à l'alignement à la limite de la voie ou la limite de fait.
- un trait discontinu ( --- ) est inscrit ou en l'absence de trait, les constructions doivent s'implanter conformément au règlement de la zone.


# 4.2. Implantation des constructions par rapport aux limites séparatives 

En complément des règles figurant ci-dessous, il convient de se reporter aux dispositions de l'article 4.2 des règlements de zone.

L'implantation des installations par rapport aux limites séparatives n'est pas règlementée.

## - Règle générale

Les balcons, les encorbellements, les éléments de décor architecturaux, les débords de toitures, les pare-soleil, les auvents et marquises ou tout autre élément sur les façades vitrées permettant la protection solaire des constructions, sont pris en compte pour le calcul de l'implantation par rapport aux limites séparatives.

Les dispositifs nécessaires à l'utilisation des énergies renouvelables ne sont pas pris en compte pour le calcul de l'implantation par rapport aux limites séparatives, à condition que leur dépassement par rapport à la façade soit inférieur à 1 m et qu'ils soient situés à une hauteur d'au moins 5 m par rapport au sol fini après travaux.

Pour l'isolation par l'extérieur des constructions existantes, se reporter aux Règles générales d'implantation par rapport aux voies, emprises publiques et limites séparatives au début de l'article 4.

## "Atlas des formes urbaines : implantations et emprises"(document graphique D1)

Lorsqu'au document graphique D1 «Atlas des formes urbaines : implantations et emprises »l'unité foncière du projet :

- est bordée par un trait continu ( $\qquad$ ), les constructions doivent s'implanter en continu, de limite à limite,
- est bordée par un trait discontinu (---), les constructions peuvent s'implanter sur une seule limite séparative ou en retrait des limites,
- n'est bordée par aucun trait, les constructions doivent s'implanter conformément au règlement de la zone.

Les prescriptions de l'atlas des formes urbaines ne s'appliquent pas aux installations.
En présence d'une ligne d'implantation ou d'une marge de recul figurant sur l'Atlas D1 des "formes urbaines: implantations et emprises", la profondeur de la bande d'implantation mentionnée aux articles 4.2 des règlements des zones UA1, UC et UCRU1, est reculée d'autant ; elle est calculée à partir de la ligne d'implantation ou de la marge de recul.
![img-6.jpeg](img-6.jpeg)
(schéma illustratif)

# 4.3. Implantation des constructions les unes par rapport aux autres sur une même propriété 

L'implantation des constructions principales les unes par rapport aux autres sur une même propriété doit permettre de préserver leur salubrité et leur éclairement et permettre l'accès des services de sécurité.

### 4.4. Emprise au sol des constructions

En complément des règles figurant ci-dessous, il convient de se reporter aux dispositions de l'article 4.4 des règlements de zone.

L'emprise au sol correspond à la projection verticale du volume de la construction, tous débords et surplombs inclus.
Sont également inclus dans l'emprise au sol :

- les installations techniques qui font partie intégrante des constructions et participent de leur volume général, tels que les ascenseurs extérieurs,
- les piscines (margelles et plages comprises),
- les constructions partiellement enterrées, avec émergence au-dessus du sol,

Sont exclus du calcul de l'emprise au sol :

- les constructions totalement enterrées même si leur emprise est supérieure à celle du reste du bâtiment
- les ornements (non clos) tels que les éléments de modénature et les auvents, marquises...
- les terrasses, les dalles végétalisées ou à usage de parking, qui ne dépassent pas le niveau du sol,

- les débords de toiture lorsqu'ils ne sont pas soutenus par des poteaux ou des encorbellements.

Lorsqu'une nouvelle construction s'implante sur une unité foncière comportant déjà des constructions, ces dernières sont prises en compte dans le calcul global de l'emprise au sol du projet : emprise des constructions existantes + emprise des constructions projetées.

Dans le cadre de la réalisation d'une opération d'aménagement d'ensemble, l'emprise au sol des constructions est calculée au regard de la totalité du terrain d'assiette du projet (espaces communs et espaces déjà bâtis compris, mais hors voies et emprises publiques).

# 4.5. Coefficient d'emprise au sol minimum et hauteur minimum au sein des périmètres d'intensification urbaine 

En complément des règles figurant ci-dessous, il convient de se reporter aux dispositions de l'article 4.5 des règlements de zone.

Les règles figurant à l'article 4.5 des règlements de zones s'appliquent aux nouvelles constructions ou opérations d'aménagement d'ensemble destinées majoritairement (en termes de surface de plancher) à l'habitation, implantées au sein d'un périmètre d'intensification urbaine défini au document graphique $E$ «Atlas des périmètres d'intensification urbaine».

En cas de mixité des destinations à l'intérieur d'un bâtiment, avec habitation majoritaire, les règles s'appliquent à la totalité du bâtiment.

Elles ne s'appliquent pas aux extensions de bâtiments existants à la date d'approbation du PLUi (le 20/12/2019).
Le coefficient d'emprise au sol (CES) au sein du périmètre d'intensification urbaine est calculé sur la base de la valeur du CES du projet X $95 \%$.

Les hauteurs minimum sont mesurées à l'égout de toiture ou au niveau de la dalle supérieure en cas de toiture terrasse.

Les règles de hauteur minimum ne s'appliquent pas :

- aux annexes et aux extensions mesurées ne dépassant pas $\underline{3,50} \mathrm{~m}$ de hauteur toiture comprise.
- en présence d'une indication sur le document graphique D2 «Atlas des formes urbaines : hauteurs» imposant une hauteur inférieure à la hauteur mini.

Compensation de la hauteur minimum, sauf en zone UCRU9 :
Un bâtiment ou une partie de bâtiment d'une hauteur inférieure à la hauteur minimum exigée peuvent être autorisés dans la mesure où la hauteur manquante peut être compensée par celle d'un autre bâtiment ou par celle d'une autre partie du même bâtiment dépassant la hauteur exigée, à condition que :

- la destination des niveaux concernés soit de l'habitation
- l'emprise au sol des niveaux concernés soit équivalente
- la compensation concerne des bâtiments situés sur la même unité foncière ou dans la même opération d'aménagement d'ensemble.

# 4.6. Hauteur des constructions et des installations 

En complément des règles figurant ci-dessous, il convient de se reporter aux dispositions de l'article 4.6 des règlements de zone.

Les règles de hauteur s'appliquent aux constructions et installations.
La hauteur totale (ou maximum) d'une construction, d'une façade, ou d'une installation correspond à la différence d'altitude entre tout point de la construction ou de l'installation et le point le plus bas situé à sa verticale.

Elle s'apprécie par rapport au niveau du terrain existant avant travaux, à la date de dépôt de la demande (se reporter à la définition du terrain existant avant travaux dans le lexique).
![img-7.jpeg](img-7.jpeg)
(schéma illustratif)
Le point le plus haut à prendre comme référence correspond au faitage de la construction, ou au sommet de l'acrotère, dans le cas de toiture-terrasse ou de terrasses en attique.

La hauteur maximale est déterminée par le règlement des zones (article 4.6), sauf règle alternative mentionnée sur le document graphique D2 «Atlas des formes urbaines : hauteurs»sur lequel la hauteur est exprimée par des traits ou des quadrillages.

Dans le périmètre d'un plan masse figurant sur le document graphique G2 «Atlas des secteurs de plan masse», les règles d'implantation et de hauteur applicables sont celles définies par le plan masse.

Dans le cadre de la mise en œuvre d'une isolation ou d'un dispositif énergétique, permettant l'amélioration de la performance énergétique du bâtiment, la surélévation de la toiture des bâtiments existants peut être autorisée dans la limite de 60 cm au-dessus du gabarit réglementaire ou en cas de réhabilitation, du gabarit du bâtiment existant à la date d'approbation du PLUI. Ces 60 cm intègrent la surélévation de 30 cm prévue par le code de l'urbanisme. Cette surélévation doit être entreprise dans le respect des dispositions architecturales originelles du bâtiment qui doivent être restituées ou adaptées.

# Sont exclus du calcul de la hauteur maximale. 

- la réhabilitation d'un bâtiment existant à la date d'approbation du PLUi (le 20/12/2019), dans le volume existant,
- sauf dans les zones UA1 et UA2 et sur les bâtiments repérés au document graphique F2 «Plan du patrimoine bâti, paysager et écologique » en niveaux 2 et 3 , les dispositifs et installations nécessaires :
- au bon fonctionnement de la construction et de faible emprise (locaux techniques d'ascenseurs, paratonnerres, souches de cheminées, dispositifs de ventilation...),
- ou à la sécurité (garde-corps)
- ou à l'agriculture urbaine et aux jardins familiaux
- ou permettant la production d'énergies renouvelables.


## «Atlas des formes urbaines : hauteurs» (document graphique D2)

Les prescriptions figurant sur cet atlas s'appliquent aux constructions (et non aux installations) et prévalent sur les règles de hauteur maxi et de hauteur par rapport aux voies, figurant dans les règlements de zone. Lorsqu'une hauteur est majorée ou minorée sur l'atlas, la hauteur à l'égout du toit mentionnée dans les règlements de zone est majorée ou minorée d'autant. Lorsqu'il est indiqué, le nombre de niveaux est majoré ou minoré d'un niveau par tranche de 3 m (si la hauteur varie de moins de 3 m le nombre de niveau ne change pas).

Lorsqu'au document graphique D2 «Atlas des formes urbaines : hauteurs»:

- un trait de couleur est inscrit, la hauteur maximale de la construction est déterminée par la couleur de ce trait et s'applique à l'ensemble de l'unité foncière.
- des hachures sont inscrites, la hauteur maximale de la construction est déterminée par la couleur de ces hachures et s'applique sur l'emprise hachurée.
- le trait est bordé par des hachures de couleur, la hauteur maximale indiquée par le trait ne s'applique que dans les 15 premiers mètres comptés depuis la limite de la voie ou la limite de fait. Au-delà c'est la hauteur déterminée par les hachures qui s'applique.

Lorsque l'unité foncière est concernée par plusieurs règles de hauteur (parcelles traversantes ...), la hauteur maximale autorisée au-delà des 15 premiers mètres bordant chaque voie, est la hauteur maximale la plus importante autorisée sur l'unité foncière.

La hauteur relative (HR) d'une construction est variable en fonction de la largeur des voies ou de la dimension des terrains.

Les balcons, les encorbellements, les éléments de décor architecturaux, les débords de toitures, les pare-soleil, les auvents et marquises ou tout autre élément sur les façades vitrées permettant la protection solaire des constructions, sont pris en compte pour le calcul de la hauteur relative.

![img-8.jpeg](img-8.jpeg)
(schémas illustratifs)

Calcul de la hauteur relative par rapport aux limites séparatives ou à l'alignement opposé
La hauteur relative est la différence d'altitude entre tout point de la construction projetée (A) et le point le plus bas et le plus proche de la limite séparative ou de l'alignement opposé (B) à l'altitude du terrain naturel (TN).

La différence d'altitude entre les points (A) et (C) représente la hauteur maxi.
![img-9.jpeg](img-9.jpeg)
(schéma illustratif)

La hauteur minimale peut être réglementée par l'article 4.5 du règlement des zones, relatif aux périmètres d'intensification urbaine. Toutefois, elle ne s'applique pas en présence d'une indication sur le document graphique D2 «Atlas des formes urbaines : hauteurs » imposant une hauteur inférieure à la hauteur mini.

# ARTICLE 5 - QUALITÉ URBAINE, ARCHITECTURALE, ENVIRONNEMENTALE ET PAYSAGÈRE 

### 5.1. Insertion des constructions et des installations dans leur environnement

En complément des règles figurant ci-dessous, il convient le cas échéant de se reporter aux dispositions de l'article 5.1 des règlements de zone.

Implantation sur le terrain et dans la pente
L'implantation de la construction doit être adaptée au terrain naturel, étudiée en fonction de la pente du terrain.
Les talus doivent être les plus longs possibles pour retrouver la pente du terrain naturel de façon progressive, sauf pour l'accès aux garages enterrés et doivent être végétalisés.

## Sont interdits :

- la création de buttes artificielles résultant d'affouillements ou d'exhaussements du sol, en vue d'y implanter une construction,
- les terrassements qui dégradent fortement le modelé naturel du site,
- les talus préfabriqués ou composés d'enrochements, qui ne sont pas compatibles avec l'OAP Paysage et biodiversité,
- les apports artificiels de terre à moins de 2 m des limites séparatives d'un terrain situé en contre-bas.

Ces interdictions ne s'appliquent pas si le projet est soumis à des prescriptions relevant de la prévention des risques, qui nécessitent la réalisation de terrassements ou s'il répond à des objectifs de réduction de la vulnérabilité face aux risques.
![img-10.jpeg](img-10.jpeg)
(schémas illustratifs)

### 5.2. Caractéristiques architecturales des façades et toitures

Il est rappelé que tous les projets doivent être conçus en compatibilité avec les orientations des OAP Paysage et biodiversité, Risques et résilience et Qualité de l'air.

Certains éléments bâtis et végétaux, repérés au document graphique F2 «Plan du patrimoine bâti, paysager et écologique »sont protégés et font l'objet, à ce titre, de règles de protection particulières. Se reporter au règlement du patrimoine, (tome 1.3 des dispositions générales du règlement).

# 5.3. Caractéristiques des clôtures 

La clôture n'est pas obligatoire ; toutefois, l'édification d'une clôture est soumise aux règles suivantes :

## - Règles générales

Les clôtures doivent être conçues de façon à ménager l'intimité au sein des parcelles tout en maintenant le caractère ouvert des espaces. Elles peuvent être composées par des haies végétales, murs, murets, murs bahut et dispositifs à claire voie (claustras...).

Leur aspect doit être choisi en fonction du contexte environnant et en compatibilité avec les orientations des OAP Risques et résilience et Paysage et biodiversité.

Les murs de clôture existants peuvent être restaurés dans la limite de leur hauteur d'origine.
Sont interdits :

- les murs de clôture pleins de plus de 1 m de haut, sauf en cas de prolongement d'un mur de clôture existant situé sur la même unité foncière ;
- les clôtures composées de palissades, de brise-vent opaques ou d'une association de matériaux hétéroclites ;
- les clôtures végétales composées d'une seule espèce ou majoritairement d'espèces persistantes (thuyas, lauriers...) ;
- Les plantes à rhizomes traçants type bambou qui prolifèrent et colonisent les alentours, sans protection spécifique (type bâche ou bac à rhizomes).
- l'emploi de matériaux tels que décrits au chapitre Aspect des Matériaux (cf art. 5.2 des règlements de zones) ;
- sauf dans les zones UCRU1, UCRU2 UCRU3, UCRU5 et UCRU9 : dans les secteurs en zone de montagne, les clôtures composées de résineux (pins, sapins, épicéas et autres espèces apparentées), pour éviter les ombres et le verglas.

Dans les espaces construits en continu, les clôtures doivent contribuer à assurer la continuité du bâti.
Côté rue et en limite de domaine public :
La hauteur de la clôture est limitée $1,80 \mathrm{~m}$. Lorsque la clôture est constituée par un muret de pierre ou de béton, surmonté d'une clôture ajourée, de préférence doublée de plantations d'essences locales et variées, la hauteur du muret ne doit pas excéder 1 mètre.
![img-11.jpeg](img-11.jpeg)

Les portillons et portails doivent être constitués de matériaux en harmonie avec ceux de la clôture ou du mur de clôture. Leur hauteur, limitée à $1,80 \mathrm{~m}$, peut exceptionnellement être augmentée si le projet s'insère dans un mur

ancien d'une hauteur supérieure, s'il s'agit de remplacer un dispositif existant ou s'il participe à la mise en valeur de la monumentalité d'une entrée existante.

# En limite séparative : 

Les clôtures doivent être réalisées avec des dispositifs pour partie perméables à la base, pour faciliter le passage de la petite faune ; leur hauteur est limitée à 2 m .

## Murs de soutènement :

Les murs de soutènement doivent être adaptés contexte urbain et au paysage urbain ou rural du territoire métropolitain : murs en maçonnerie enduite, en petit appareil de type pierres sèches, en béton structuré avec motifs ou reliefs ; les enrochements sont interdits.

Les murs de soutènement en gabions sont autorisés à condition que les cages métalliques soient les plus discrètes possibles et remplies avec des pierres s'adaptant au paysage en forme et en couleur dans l'esprit d'appareillage traditionnel de mur. Les remplissages en vrac avec des galets dont le rendu ne respecte pas les caractéristiques locales sont proscrits.

Ces murs seront végétalisés dans la mesure du possible.
L'usage des gabions doit être réservé à des usages de soutènement.
![img-12.jpeg](img-12.jpeg)

Lorsqu'une clôture est réalisée sur un mur de soutènement, la hauteur du mur de soutènement n'est pas comprise dans le calcul de la hauteur de la clôture.

Par rapport au niveau du point haut du mur de soutènement, la clôture ne peut toutefois pas excéder :

- 1,5 m en limite du domaine public
- 1,8 m en limite séparative.

![img-13.jpeg](img-13.jpeg)
(schéma illustratif)

# - Règles alternatives 

Des dispositions autres que celles prévues par les règles générales peuvent être imposées :

- pour les équipements d'intérêt général et les services publics ;
- pour la réalisation d'un équipement ou d'une installation technique liés à la sécurité, à l'accessibilité d'un bâtiment (ascenseur, escaliers), aux différents réseaux, voirie et stationnement ou nécessaires à la production d'énergies renouvelables ;
- pour la préservation ou la restauration d'un élément architectural ou végétal protégé et repéré sur le document graphique F2 «Plan du patrimoine bâti, paysager et écologique»;
- lorsque des dispositifs spécifiques doivent être mis en place pour garantir la sécurité publique ou éviter un trouble pour le voisinage :
- soit en raison de l'activité exercée sur le terrain,
- soit du fait de fortes nuisances sonores générées par le trafic automobile ou ferroviaire, c'est-à-dire dans les secteurs affectés par les voies classées en catégorie 1 et 2 par RFF (Réseau ferré de France) dans l'atlas des nuisances sonores (cf. Annexe 3-c).


### 5.4. Prescriptions relatives au patrimoine bâti et paysager à protéger, à conserver, à restaurer, à mettre en valeur ou à requalifier

Les constructions et installations à édifier ou à modifier doivent participer, par leur situation, leur architecture, leurs dimensions ou leur aspect extérieur, à l'intérêt et à la mise en valeur du caractère des lieux avoisinants, des sites, des paysages naturels ou urbains, et à la conservation des perspectives monumentales.

Certains éléments bâtis et végétaux, repérés au document graphique F2 «Plan du patrimoine bâti, paysager et écologique »sont protégés et font l'objet à ce titre, de règles de protection particulières (cf règlement du patrimoine, Tome 1.3 des dispositions générales du règlement).

Doivent satisfaire aux conditions fixées par ces règles particulières :

- la démolition, la transformation, l'extension ou le changement de destination des éléments et ensembles bâtis repérés au document graphique F2 «Plan du patrimoine bâti, paysager et écologique»

- la destruction ou la modification des éléments, linéaires et ensembles végétaux repérés au document graphique F2 «Plan du patrimoine bâti, paysager et écologique».


# ARTICLE 6 - TRAITEMENT ENVIRONNEMENTAL ET PAYSAGER DES ESPACES NON BÂTIS, DES CONSTRUCTIONS ET DE LEURS ABORDS 

### 6.1. Obligations en matière de réalisation d'espaces libres et de plantations, d'aires de jeux et de loisirs

Se reporter aux dispositions de l'article 6.1 des règlements de zone.

### 6.2. Surfaces végétalisées ou perméables

## - Rappel

En complément des règles figurant ci-dessous, il convient de se reporter aux dispositions des règlements de zone.
Certains végétaux et parcs repérés au document graphique F2 «Plan des patrimoines bâti, paysager et écologique» sont protégés et font l'objet, à ce titre, de règles de protection particulières (cf. règlement du patrimoine, Tome 1.3 des dispositions générales du règlement).

## - Règles générales

Les règles de végétalisation s'appliquent au terrain ou à l'unité foncière. Elles sont mises en œuvre en compatibilité avec les orientations de l'OAP Paysage et biodiversité.

Les surfaces végétalisées peuvent être utilisées comme support d'une agriculture urbaine ou comme jardins collectifs, avec aménagements paysagers et plantation d'arbres et arbustes à vocation alimentaire.

## Pleine terre :

Les espaces de pleine terre doivent être réalisés majoritairement (+ de 50\%) d'un seul tenant et avoir des proportions permettant un usage d'agrément et de faire des plantations, selon les caractéristiques du terrain et de son environnement. Afin d'éviter les plantes invasives, les espaces de pleine terre doivent être plantés et il est interdit de laisser le sol nu, non végétalisé.
![img-14.jpeg](img-14.jpeg)
(schéma illustratif)

Pour tout espace de pleine terre d'une surface supérieure ou égale à $100 \mathrm{~m}^{2}$, au moins un arbre de haute tige, sera planté par tranche de $100 \mathrm{~m}^{2}$ de pleine terre, en se référant aux modalités de plantation et au choix des essences présentées dans l'OAP Paysage et Biodiversité. Les arbres existants, conservés dans le projet et situés sur l'espace de pleine terre peuvent être comptabilisés. Ces dispositions ne s'appliquent pas dans les zones A, AL, N et NL.

Sont pris en compte pour la quantification des espaces de pleine terre :

- Les espaces de terre végétale, (dont l'épaisseur sera adaptée à la typologie des plantations), libres, non couverts, non bâtis ni en surface ni en sous-sol, permettant la libre infiltration des eaux pluviales et aménagés en espaces verts (pelouses, plantations) ;
- Les espaces de terre végétale situés au-dessus des canalisations et des bassins d'infiltration réalisés sans ouvrages.

Ne sont pas comptabilisés en espaces de pleine terre :

- Les surfaces situées au pied des arbres isolés, imposés sur les aires de stationnement et inférieures aux diamètres de fosses de plantation décrits dans l'OAP Paysage et Biodiversité, diamètres qui varient selon la taille des arbres et de l'ambiance .
- les espaces de circulation et de manœuvre des véhicules, notamment les espaces enherbés situés entre les roues des véhicules ;
- Les espaces de terre situés sur des bassins de rétention étanches ;
- Les mélanges terre-pierre.


# Espaces végétalisés ou perméables 

- Les espaces végétalisés ou perméables comprennent :
- les espaces de pleine terre précédemment mentionnés (le pourcentage de pleine terre peut donc être inclus dans le pourcentage d'espace végétalisé),
- les mélanges terre-pierre,
- les espaces végétalisés situés sur les dalles de couverture,
- les toitures végétalisées,
- les surfaces des façades végétalisées,
- les espaces végétalisés structurants des aires de stationnement ; leur coefficient de pondération sera choisi selon les modalités de réalisation : pleine terre ou sur dalle ;
- les espaces extérieurs réalisés en matériaux perméables ou en matériaux semi-perméables avec revêtement pour partie minéral : cheminements piétons, circulations, aires de jeux, stationnements.
- la végétation mise en place sur les parties communes des constructions (hors parties privatives telles que balcons, terrasses des logements...).

[^0]
[^0]:    - Les espaces végétalisés ou perméables ne comprennent pas :
    - Les surfaces extérieures avec revêtement non perméables.

# - Les jardinières. 

## Coefficient de pondération

Le coefficient de pondération est appliqué comme indiqué (cf. le tableau ci-dessous).

| Type d'espace | Coefficient de pondération |
| :-- | :-- |
| Espaces de pleine terre | 1,0 |
| Espaces végétalisés sur dalles de couverture et toitures : <br> épaisseur $\geq 50 \mathrm{~cm}$ <br> 20 s épaisseur < 50 cm <br> épaisseur < 20 cm | 0,7 |
| Surface des façades végétalisées : | 0,5 |
| Espaces extérieurs réalisés en matériaux perméables ou en matériaux semi-perméables <br> avec revêtement pour partie minéral, hors mélange terre-pierres : cheminements piétons, <br> circulations, aires de jeux, stationnements. | 0,3 |
| Mélange terre-pierres : | 0,2 |

Le coefficient de surfaces végétalisées ou perméables est calculé de la manière suivante :
$\operatorname{Coef}[\mathrm{SV}]=\frac{\operatorname{Surf}[\mathrm{PT}]+(\operatorname{Surf}[\mathrm{esp} 1] \times \text { Pond [esp1] })+(\operatorname{Surf}[\mathrm{esp} 2] \times \text { Pond [esp2] })+\ldots}{\text { Surface totale du terrain }}$
Pond [esp1] = coefficient de pondération de l'espace 1
Surf [esp2] = surface d'espace 2
Pond [esp2] = coefficient de pondération de l'espace 2
avec Surf [PT] = surface de pleine terre
Surf [esp1] = surface d'espace 1

![img-15.jpeg](img-15.jpeg)
(Schéma illustratif)

Formule correspondant au schéma:

$$
\text { Coef }[\mathrm{SV}]=\frac{100 \mathrm{~m}^{2}[\mathrm{PT}]+\left\{65 \mathrm{~m}^{2}[\text { toiture végétalisée }] \times 0,5\right\}+\left\{11,5 \mathrm{~m}^{2}[\text { espace de stationnement réalisé en }\right.}{\text { matériau perméable }] \times 0,2\}+\left\{18,5 \mathrm{~m}^{2}[\text { espace végétalisé structurant aire stationnement }] \times 0,5\right\}}
$$

$$
200 \mathrm{~m}^{2} \text { [surface totale du terrain] }
$$

$$
\text { Coef }[\mathrm{SV}]=\frac{144 \mathrm{~m}^{2}}{200 \mathrm{~m}^{2}}=0,72
$$

Les dispositions prévues par la règle générale ne s'appliquent pas :

- en cas d'extension mesurée d'une construction existante, sous réserve de justifications techniques, architecturales, d'intégration du projet dans le site ou dans son contexte ;
- en cas de réhabilitation ou de changement de destination d'une construction existante dans le volume existant ;
- en cas de construction d'une annexe inférieure ou égale à $20 \mathrm{~m}^{2}$ de surface de plancher et/ou d'emprise au sol ;
- pour les équipements d'intérêt général et les services publics, notamment lorsque l'usage du terrain ne permet pas de le végétaliser, sauf dans les zones UV et UZ;
- pour la réalisation d'un équipement ou d'une installation technique liés à la sécurité, à l'accessibilité d'un bâtiment (ascenseur, escaliers), ou nécessaires à la production d'énergies renouvelables ;
- en cas de travaux sur un bâtiment protégé et repéré au document graphique F2 «Plan du patrimoine bâti, paysager et écologique »;
- en cas de travaux sur un bâtiment protégé au titre des sites inscrits ou classés ou au titre des monuments historiques.

# - Règles alternatives 

En cas d'extension d'une construction existante et en cas d'impossibilité technique avérée de réaliser les pourcentages d'espace végétalisés ou perméables imposés par la règle générale :

- il n'est pas imposé de réaliser d'espace de pleine terre ;
- la surface végétalisée ou perméable requise doit être au moins équivalente à l'emprise au sol bâtie créée, en tenant compte des coefficients de pondération définis ci-dessus.

Dans le cadre de la réalisation d'une opération d'aménagement d'ensemble, le pourcentage d'espaces végétalisés ou perméables peut être calculé au regard de la totalité de l'opération (espaces communs et espaces déjà bâtis compris, mais hors voiries), à condition que la répartition de la végétalisation au sein dudit périmètre permette de respecter l'obligation globale.

Lorsqu'un pourcentage d'espaces végétalisés et/ou de pleine terre figure sur le document graphique D1- "Atlas des formes urbaines - implantations et emprises" ce pourcentage s'impose par rapport à celui figurant dans le règlement de zone.

Les prescriptions de l'atlas des formes urbaines ne s'appliquent pas aux installations.

### 6.3. Maintien ou remise en état des continuités écologiques

## - Protection des zones humides

Sont interdits dans les zones humides délimitées au document graphique F2 «Plan du patrimoine bâti, paysager et écologique»:

- Le drainage, les remblaiements et déblaiements, le dépôt ou l'extraction de matériaux, à l'exception des travaux et ouvrages nécessaires au maintien en l'état ou à la régulation de l'alimentation en eau de la zone humide, ainsi qu'à la protection contre les risques naturels.
- L'imperméabilisation totale ou partielle du sol.
- Toute construction et installation nouvelle, usage et affectation du sol à l'exception :
- de ceux liés à la mise en valeur ou à l'entretien du milieu ;
- des serres-tunnels et des tunnels agricoles ;
- de la réhabilitation dans le volume existant des constructions existantes.


## - Préservation des berges des cours d'eau ou des fossés

Toute construction nouvelle ou toute extension de construction doit respecter un recul minimum par rapport au haut de la berge :

Dans les zones urbaines mixtes (U) :

- de 5 mètres

Dans les zones A et N :
Pour les cours d'eau ou fossés repérés sur le document graphique F2 «Plan du patrimoine bâti, paysager et écologique», le recul est de :

- 15 mètres pour les cours d'eau ou fossés repérés au titre de la Trame Verte et Bleue
- 10 mètres pour les autres cours d'eau et fossés.

Sauf dispositions contraires des titres II et III du règlement des risques, les locaux techniques et industriels des administrations publiques et assimilés, ne sont pas soumis à ces exigences si, du fait de leur nature ou pour des raisons techniques, il est nécessaire de les rapprocher des berges.

Pour les lits mineurs couverts (galerie, busage etc.), le recul minimum à appliquer correspond à la largeur du lit couvert plus 2 m de part et d'autre.

# 6.4. Gestion des eaux pluviales et du ruissellement 

Les aménagements extérieurs des constructions doivent contribuer à limiter l'imperméabilisation des sols et la réutilisation des eaux pluviales doit être privilégiée dans la conception et la réhabilitation des constructions.

## - Prescriptions relatives aux espaces de bon fonctionnement des zones humides

Dans les espaces de bon fonctionnement des zones humides délimités sur le document graphique F2 «Plan du patrimoine bâti, paysager et écologique », les constructions ne peuvent être autorisées qu'à condition :

- de ne pas détourner les eaux reçues et de les restituer à l'espace de fonctionnalité de la zone humide ;
- en cas de drainage, de rejeter les eaux drainées dans l'espace de fonctionnalité de la zone humide.


### 6.5. Aménagement d'emplacements spécifiques dédiés à la collecte des déchets ménagers et assimilés

En complément des règles figurant ci-dessous, il convient de se reporter aux dispositions des règlements de zone.

## - Règle générale

Dans les secteurs concernés par le système de collecte en porte à porte, tout projet de construction doit comprendre l'aménagement sur l'unité foncière d'un emplacement de présentation des conteneurs individuels ou collectifs, sauf impossibilité pour des raisons architecturales ou patrimoniales (dent creuse dans le tissu urbain dense des centresanciens, villages et hameaux), d'impossibilité technique avérée, ou dans le cas d'une mutualisation du point de collecte pour plusieurs bâtiments.

Cet emplacement doit être directement accessible depuis le domaine public ou depuis les voies ouvertes à la circulation des poids lourds et être dimensionné de manière à répondre aux préconisations techniques du cahier de recommandations techniques de collecte (disponible sur le site web de la Métro).

Lorsque l'opération est desservie par une voie ou un chemin privés communs à plusieurs parcelles, cet emplacement doit être prévu au débouché sur la voie publique.

Lorsque les locaux et aires de présentation servent également de lieu de stockage permanent des conteneurs à déchets, ils doivent être traités de façon à réduire leur impact visuel par une haie compacte ou tout autre dispositif adapté à l'environnement du site, tels que murets ou panneaux à claire-voie.

Dans tous les cas, la localisation et le dimensionnement de l'aire devront être conçus de manière à prendre en compte le règlement technique de collecte (disponible sur le site web de la Métro).

# CHAPITRE 3 - EQUIPEMENTS ET RESEAUX 

## ARTICLE 7 - STATIONNEMENT

### 7.1. Stationnement des véhicules motorisés

### 7.1.1. Principales caractéristiques des aires de stationnement

En complément des règles figurant ci-dessous, il convient de se reporter aux dispositions des règlements de zone.

## - Localisation des places

Les places de stationnement doivent être réalisées en dehors des voies publiques et peuvent être situées sur l'assiette foncière de l'opération ou dans son environnement immédiat.

Lorsque le pétitionnaire ne peut pas satisfaire aux obligations résultant du premier alinéa, il peut être tenu quitte de ces obligations en justifiant, pour les places qu'il ne peut réaliser lui-même :

- soit de l'obtention d'une concession à long terme dans un parc public de stationnement existant ou en cours de réalisation et situé à proximité de l'opération,
- soit de l'acquisition ou de la concession de places dans un parc privé de stationnement répondant aux mêmes conditions.


## - Caractéristiques des places de stationnement

Les dimensions minimales d'une place de parking sont de 2,3 m de large et 5 m de long.
Cette disposition ne s'applique pas lorsque les places sont aménagées parallèlement aux voies de circulation.

## - Accès aux espaces de stationnement depuis la voirie publique

Lorsque l'opération prévoit la réalisation de plus de 50 places de stationnement, les accès au parking doivent intégrer un espace permettant aux véhicules d'attendre l'ouverture des barrières ou du portail d'accès en dehors de la voirie publique.

Lorsque l'occupation future de l'opération nécessite l'accès à des véhicules de gros gabarit (poids lourds, bus...), le projet doit intégrer les mesures nécessaires pour limiter leur impact sur la circulation.

## - Aménagement paysager et lutte contre l'imperméabilisation des sols

Les places de stationnement doivent être de préférence intégrées au bâtiment.
Toutefois, lorsque les places sont réalisées en surface et non couvertes, au moins $30 \%$ de la surface dédiée au stationnement (places et circulations) doit recevoir un traitement paysager permettant d'infiltrer les eaux pluviales et/ ou de végétaliser.

Sauf en zone ALv6, les aires de stationnement extérieures doivent être plantées d'arbres de haute tige à raison d'au moins un arbre pour 3 places de stationnement.

Les plantations doivent être :

- réparties sur l'ensemble de l'aire de stationnement de manière à ombrager les places,
- ou être regroupées en un ou plusieurs bosquets.

Les essences locales variées, d'espèces buissonnantes et majoritairement à feuilles caduques, doivent être favorisées.
Les arbres de haute tige existants et conservés peuvent être comptabilisés dans le nombre d'arbres à planter à condition qu'ils soient situés sur l'aire de stationnement.

# - Stationnement et production d'énergies renouvelables 

Lorsque le projet prévoit la réalisation de stationnement pour véhicules légers en extérieur d'une surface supérieure à $1000 \mathrm{~m}^{2}$ en une seule ou plusieurs aires (places hors circulations), il doit être prévu la production d'au moins 50 kWhef par $\mathrm{m}^{2}$ de surface de stationnement (places véhicules légers hors circulations) et par an.

Les équipements nécessaires à cette production peuvent être regroupés sur certains bâtiments et/ou parkings au regard de leur localisation et/ou orientation.

## - Places adaptées aux personnes à mobilité réduite

Pour les bâtiments collectifs d'habitation, au moins $5 \%$ du total des places de stationnement à réaliser doivent être accessibles aux personnes à mobilité réduite. Les caractéristiques de ces places doivent répondre aux obligations règlementaires en la matière.

### 7.1.2. Nombre de places à réaliser

Le nombre de places de stationnement à réaliser dépend :

- de la localisation du projet par rapport aux secteurs du zonage stationnement. Ce zonage est défini par le document graphique $H$ «Atlas du stationnement»;
- de la ou des catégorie(s), destination(s) et sous-destination(s) du projet.

Le nombre de places à réaliser est arrondi à l'entier le plus proche. L'entier et demi est arrondi à l'entier supérieur. En cas de norme a minima, la réalisation d'au moins une place de stationnement est imposée.

En cas de juxtaposition ou d'imbrication de plusieurs destinations/sous-destinations, les obligations relatives à chaque destination/sous-destination se cumulent.

Si l'unité foncière d'un projet est à cheval sur plusieurs secteurs définis par le document graphique $H$ «Atlas du stationnement», les normes et règles applicables en matière de stationnement sont celles du secteur qui s'applique sur la plus grande partie de l'unité foncière du projet.

Lorsque la norme ne fixe pas un ratio mais une réalisation selon les besoins de l'opération, l'estimation des besoins doit être justifiée par le pétitionnaire et tenir compte :

- du nombre de futurs usagers du projet,
- du niveau de desserte de l'opération par les transports en commun et le réseau structurant cycles, les espaces situés dans les zones S1, S2, S3 et S5 étant considérés comme bien desservis,

- des conditions de stationnement existantes ou à créer (parkings publics, stationnement sur voirie, etc...) à proximité de l'opération (à moins de 500 m ),
- des possibilités de mutualisation des aires de stationnement existantes à proximité (à moins de 500 m ) ou prévues au sein de l'opération,
- des critères de mobilité des futurs usagers du projet (besoins, horaires et types de déplacements, existence de Plans de Mobilité...).

Pour les opérations de bureaux ou de locaux et bureaux accueillant du public des administrations publiques et assimilés, le maximum fixé par le règlement pourra être exceptionnellement dépassé lorsque l'activité le justifiera (besoin d'une importante flotte de véhicules d'entreprise, accueil de nombreux visiteurs externes).

Normes de stationnement pour les véhicules motorisés

| Catégories <br> Destinations <br> Sous-destinations <br> (sous-réserve qu'elles soient <br> autorisées dans la zone) | Secteur S1 | Secteur S2 | Secteur S3 |
| :--: | :--: | :--: | :--: |
| Logement social | 0,25 place minimum par <br> logement | 0,5 place minimum par <br> logement | 0,7 place minimum par <br> logement |
| Logement très social <br> (PLAI ou niveau de <br> loyers équivalent) | Pas d'obligation |  |  |
| Autres logements | 1 place minimum pour <br> $140 \mathrm{~m}^{2}$ de SP* | Opération $<150 \mathrm{~m}^{2}$ de <br> SP* : 1 place minimum <br> Opération $>150 \mathrm{~m}^{2}$ de <br> SP* : 1 place pour les <br> premiers $150 \mathrm{~m}^{2}$ de SP*, <br> puis 1 place minimum <br> pour $80 \mathrm{~m}^{2}$ de SP* | Opération $<150 \mathrm{~m}^{2}$ de SP* : <br> 1 place minimum <br> Opération $>150 \mathrm{~m}^{2}$ de SP*: <br> 1 place pour les premiers <br> $150 \mathrm{~m}^{2}$ de SP*, puis 1 place <br> minimum pour $70 \mathrm{~m}^{2}$ de <br> SP* |
|  | Il ne pourra être exigé plus d'une place de stationnement par <br> logement <br> * SP : Surface de plancher affectée au logement (hors logement social) |  |  |
| Résidences <br> universitaires | Pas d'obligation | 0,15 place minimum par <br> place d'hébergement <br> Sauf dans les zones UZ2 <br> indicées « g » dans <br> lesquelles il n'est fait état <br> d'aucune obligation de <br> réalisation d'aires de <br> stationnement | 0,3 place minimum par <br> place d'hébergement |
| Artisanat et commerce <br> de détail | 1 place maximum pour <br> $75 \mathrm{~m}^{2}$ de Surface de <br> Vente | 1 place maximum pour $40 \mathrm{~m}^{2}$ de Surface de Vente |  |
| Restauration / <br> Activités de services <br> où s'effectue l'accueil <br> d'une clientèle | 1 place maximum pour <br> $95 \mathrm{~m}^{2}$ de Surface de <br> Plancher | 1 place maximum pour $50 \mathrm{~m}^{2}$ de Surface de Plancher |  |
| Bureaux / <br> Locaux et bureaux <br> accueillant du public <br> des administrations <br> publiques et assimilés | 1 place maximum pour <br> $60 \mathrm{~m}^{2}$ de Surface de <br> Plancher | 1 place maximum pour <br> $50 \mathrm{~m}^{2}$ de Surface de <br> Plancher | 1 place maximum pour <br> $40 \mathrm{~m}^{2}$ de Surface de <br> Plancher |
| Autres destinations et <br> sous-destinations | Selon les besoins de l'opération |  |  |

Normes de stationnement pour les véhicules motorisés

| Catégories <br> Destinations <br> Sous-destinations <br> (sous réserve qu'elles soient <br> autorisées dans la zone) | Secteur S4 | Secteur S5 |
| :--: | :--: | :--: |
| Logement social | 1 place minimum par logement | 0,8 place minimum par logement |
| Logement très social <br> (PLAI ou niveau de <br> loyers équivalent) |  |  |
| Autres logements | Opération $\leq 60 \mathrm{~m}^{2}$ de SP*: 1 place <br> minimum <br> Opération $>60 \mathrm{~m}^{2}$ et $\leq 150 \mathrm{~m}^{2}$ de SP*: 2 <br> places minimum <br> Opération $>150 \mathrm{~m}^{2}$ de SP*: 2 places <br> pour les premiers $150 \mathrm{~m}^{2} \mathrm{SP}^{*}$, puis 1 <br> place minimum pour $60 \mathrm{~m}^{2}$ de SP | Opération $\leq 150 \mathrm{~m}^{2}$ de SP*: 1 place <br> minimum <br> Opération $>150 \mathrm{~m}^{2}$ de SP*: 1 place <br> pour les premiers $150 \mathrm{~m}^{2}$ de SP*, puis 1 <br> place minimum pour $70 \mathrm{~m}^{2}$ de SP |
| Résidences <br> universitaires | 0,3 place minimum par place d'hébergement |  |
| Artisanat et commerce <br> de détail | Opération <400m² de Surface de <br> Vente: pas d'obligation <br> Opération $>400 \mathrm{~m}^{2}$ et < 1000m² de <br> Surface de Vente: 1 place minimum <br> pour $50 \mathrm{~m}^{2}$ de Surface de Vente <br> Opération $>1000 \mathrm{~m}^{2}$ de Surface de <br> Vente: 1 place minimum pour $30 \mathrm{~m}^{2}$ de <br> Surface de Vente | 1 place maximum pour $30 \mathrm{~m}^{2}$ de <br> Surface de Vente |
| Restauration / <br> Activités de services où <br> s'effectue l'accueil <br> d'une clientèle | Opération <500m² de Surface de <br> Plancher: pas d'obligation <br> Opération $>500 \mathrm{~m}^{2}$ et < 1250m² de <br> Surface de Plancher: 1 place minimum <br> pour $60 \mathrm{~m}^{2}$ de Surface de Plancher <br> Opération $>1250 \mathrm{~m}^{2}$ de Surface de <br> Plancher: 1 place minimum pour $40 \mathrm{~m}^{2}$ <br> de Surface de Plancher | 1 place maximum pour $40 \mathrm{~m}^{2}$ de <br> Surface de Plancher |
| Bureaux/ <br> Locaux et bureaux <br> accueillant du public <br> des administrations <br> publiques et assimilés | 1 place minimum pour $60 \mathrm{~m}^{2}$ de <br> Surface de Plancher | 1 place maximum pour $30 \mathrm{~m}^{2}$ de <br> Surface de Plancher |
| Autres destinations et <br> sous-destinations | Selon les besoins de l'opération |  |

Normes de stationnement pour les véhicules motorisés

| Catégories <br> Destinations <br> Sous-destinations <br> Nous réserve qu'elles soient <br> survenées dans la zone | Secteur S6 | Secteur SC |
| :--: | :--: | :--: |
| Logement social | 1 place minimum par logement |  |
| Logement très social <br> (PLAI ou niveau de <br> loyers équivalent) |  |  |
| Autres logements | Opération $<60 \mathrm{~m}^{2}$ de SP*: 1 place <br> minimum <br> Opération $>60 \mathrm{~m}^{2}$ et $<150 \mathrm{~m}^{2}$ de SP*: 2 <br> places minimum <br> Opération $>150 \mathrm{~m}^{2}$ de SP*: 2 places <br> pour les premiers $150 \mathrm{~m}^{2} \mathrm{SP}^{*}$, puis 1 <br> place minimum pour $45 \mathrm{~m}^{2}$ de SP | 1 place minimum pour $80 \mathrm{~m}^{2}$ de SP |
| Résidences universitaires | 0,3 place minimum par place d'hébergement | Non concerné |
| Artisanat et commerce de détail | Opération $<400 \mathrm{~m}^{2}$ de Surface de <br> Vente; pas d'obligation <br> Opération $>400 \mathrm{~m}^{2}$ et $<1000 \mathrm{~m}^{2}$ de <br> Surface de Vente: 1 place minimum pour $50 \mathrm{~m}^{2}$ de Surface de Vente <br> Opération $>1000 \mathrm{~m}^{2}$ de Surface de <br> Vente: 1 place minimum pour $30 \mathrm{~m}^{2}$ de Surface de Vente | 1 place minimum pour $30 \mathrm{~m}^{2}$ de <br> Surface de Vente |
| Restauration / <br> Activités de services où s'effectue l'accueil d'une clientèle | Opération $<500 \mathrm{~m}^{2}$ de Surface de <br> Plancher: pas d'obligation <br> Opération $>500 \mathrm{~m}^{2}$ et $<1250 \mathrm{~m}^{2}$ de <br> Surface de Plancher: 1 place minimum pour $60 \mathrm{~m}^{2}$ de Surface de Plancher <br> Opération $>1250 \mathrm{~m}^{2}$ de Surface de <br> Plancher: 1 place minimum pour $40 \mathrm{~m}^{2}$ de Surface de Plancher | 1 place minimum pour $40 \mathrm{~m}^{2}$ de <br> Surface de Plancher |
| Bureaux/ <br> Locaux et bureaux accueillant du public des administrations publiques et assimilés | 1 place minimum pour $50 \mathrm{~m}^{2}$ de <br> Surface de Plancher | 1 place maximum pour $30 \mathrm{~m}^{2}$ de <br> Surface de Plancher |
| Autres destinations et sous-destinations | Selon les besoins de l'opération |  |

# 7.1.3. Dispositions particulières 

En complément des règles figurant ci-dessous, il convient de se reporter aux dispositions des règlements de zone.

## - Opérations d'aménagement d'ensemble

Dans le cadre de la réalisation d'une opération d'aménagement d'ensemble, les obligations de réalisation des aires de stationnement pour les véhicules motorisés ainsi que les obligations de production d'énergies renouvelables peuvent être appréciées à l'échelle de l'opération.

## - Commerces soumis à autorisation commerciale

Pour les commerces soumis à l'autorisation d'exploitation commerciale prévue aux $1^{\circ}$ et $4^{\circ}$ du I de l'article L. 752-1 du code de commerce et à l'autorisation prévue au $1^{\circ}$ de l'article L. 212-7 du code du cinéma et de l'image animée, les obligations de réalisation d'aires de stationnement doivent respecter les dispositions de l'article L.111-19 du Code de I'Urbanisme.

## - Mutualisation des aires de stationnement, autopartage et nouvelles mobilités

Quand le projet comporte plusieurs destinations ou sous-destinations permettant la mutualisation de tout ou partie des aires de stationnement, le nombre global de places à réaliser peut être réduit de $20 \%$.

Pour les opérations à usage d'habitation, les obligations de réalisation d'aires de stationnement pour les véhicules motorisés peuvent être réduites de $15 \%$ en contrepartie de la mise à disposition de véhicules propres en autopartage.

Dans les secteurs 52 et 53 du zonage stationnement défini par le document graphique $H$ «Atlas du stationnement», les opérations d'artisanat et de commerces de détail de plus de $1000 \mathrm{~m}^{2}$ de surface de vente peuvent réaliser 1 place maximum pour $30 \mathrm{~m}^{2}$ de surface de vente si le pétitionnaire justifie :

- de la mutualisation possible des places de stationnement, permettant aux riverains d'utiliser le parking en dehors des heures d'ouverture du commerce ;
- ou de l'équipement d'au moins $20 \%$ des places de bornes de recharges pour les véhicules électriques ou hybrides.


### 7.2. Stationnement des cycles

### 7.2.1. Principales caractéristiques des aires de stationnement

## - Localisation des places

Les places de stationnement pour les cycles doivent être réalisées en dehors des voies publiques et situées sur l'assiette foncière de l'opération.

Les places de stationnement pour les cycles doivent être réalisées à l'intérieur des bâtiments principaux de l'opération ou à l'extérieur de ces derniers sous réserve d'être localisées à moins de 50 m de l'une de leurs entrées principales. Ces places doivent être situées de préférence en rez-de-chaussée et être aisément accessibles depuis les voies publiques.

Pour les bâtiments à destination principale de bureau, d'industrie ou d'habitation groupant au moins deux logements/ hébergements, l'espace de stationnement des vélos peut être réalisé en extérieur à condition d'être couvert et clos.

Pour les autres destinations et en complément des prescriptions du présent règlement, la réalisation des places de stationnement vélo doit répondre aux caractéristiques définies aux articles R.111-14-4 à R.111-14-8 du Code de la construction et de l'habitation.

# - Dimension des places et des locaux 

La dimension minimale d'un emplacement cycle est de $1,5 \mathrm{~m}^{2}$. Cette surface intègre les espaces de circulation nécessaires au bon fonctionnement de l'aire de stationnement des cycles.

Tout en conservant la surface de stationnement vélo exigée au 7.2.2, le nombre d'emplacements demandé peut être réduit si l'opération prévoit l'aménagement d'emplacements pour vélocargos ou pour les vélos avec remorque.

### 7.2.2. Nombre de places à réaliser

Le nombre de places de stationnement à réaliser dépend :

- de la localisation du projet par rapport aux secteurs du zonage stationnement, définis par le document graphique H «Atlas du stationnement»;
- de la ou des catégorie(s), destination(s) et sous-destination(s) dans le projet.

Le nombre de places à réaliser est arrondi à l'entier le plus proche. L'entier et demi est arrondi à l'entier supérieur.

Si l'unité foncière d'un projet est à cheval sur plusieurs secteurs définis par le document graphique $H$ «Atlas du stationnement», les normes et règles applicables en matière de stationnement sont celles du secteur concerné par la plus grande partie de l'unité foncière du projet.

Lorsque la norme ne fixe pas un ratio mais une réalisation selon les besoins de l'opération, l'estimation des besoins doit être justifiée par le pétitionnaire en tenant compte :

- du nombre de futurs usagers du projet,
- du niveau de desserte de l'opération par les transports en commun et le réseau structurant cycles, les espaces situés dans les zones S1, S2, S3 et S5 étant considérés comme bien desservis,
- des conditions de stationnement existantes ou à créer (parkings publics, etc...) à proximité de l'opération (à moins de 500 m ),
- des possibilités de mutualisation des aires de stationnement existantes à proximité (à moins de 500 m ) ou prévues au sein de l'opération,
- des critères de mobilité des futurs usagers du projet (besoins, horaires et types de déplacements, existence de Plans de Déplacements des Entreprises ou des Administrations...).

Normes de stationnement pour les vélos

| Catégories <br> Destinations <br> Sous-destinations <br> (sous réserve qu'elles soient <br> autorisées dans la zone) | Secteur S1 | Secteur S2 | Secteur S3 |
| :-- | :-- | :-- | :-- |
| Logement | 2 emplacements jusqu'à $150 \mathrm{~m}^{2}$ de Surface de Plancher <br> Au-delà, $\mathbf{2}$ emplacements pour les premiers $150 \mathrm{~m}^{2}$ de Surface de Plancher, <br> puis 1 emplacement minimum pour $35 \mathrm{~m}^{2}$ de Surface de Plancher |  |  |
| Résidences <br> universitaires | 1 emplacement minimum par place d'hébergement |  |  |
| Artisanat et <br> commerce de détail | Pas d'obligation | Opération <400m² de Surface de Vente : pas <br> d'obligation <br> Opération $\geq 400 \mathrm{~m}^{2}$ de Surface de Vente : Si <br> réalisation de places de stationnement pour les <br> voitures, alors 10 emplacements vélos +1 <br> emplacement vélo/5 places de stationnement <br> réalisées pour les voitures |  |
| Restauration / <br> Activités de services <br> où s'effectue l'accueil <br> d'une clientèle | Pas d'obligation | Opération <500m² de Surface de Plancher : pas <br> d'obligation <br> Opération $\geq 500 \mathrm{~m}^{2}$ de Surface de Plancher : Si <br> réalisation de places de stationnement pour les <br> voitures, alors 10 emplacements vélos +1 <br> emplacement vélo/5 places de stationnement <br> réalisées pour les voitures |  |
| Bureaux/ <br> Locaux et bureaux <br> accueillant du public <br> des administrations <br> publiques et <br> assimilés | 1 emplacement <br> minimum pour $45 \mathrm{~m}^{2}$ <br> de Surface de Plancher | 1 emplacement minimum pour $60 \mathrm{~m}^{2}$ de Surface <br> de Plancher |  |
| Autres destinations et <br> sous-destinations | Selon les besoins de l'opération |  |  |

# Normes de stationnement pour les vélos 

| Catégories <br> Destinations <br> Sous-destinations <br> (sous réserve qu'elles soient <br> autorisées dans la zone) | Secteur S4 | Secteur S5 |
| :--: | :--: | :--: |
| Logement | 2 emplacements jusqu'à $150 \mathrm{~m}^{2}$ de Surface de Plancher Au-delà, 2 emplacements pour les premiers $150 \mathrm{~m}^{2}$ de Surface de Plancher, puis 1 emplacement minimum pour $35 \mathrm{~m}^{2}$ de Surface de Plancher |  |
| Résidences universitaires | 1 emplacement minimum par place d'hébergement |  |
| Artisanat et commerce de détail | Opération <400m² de Surface de Vente : pas d'obligation <br> Opération $\geq 400 \mathrm{~m}^{2}$ de Surface de Vente : $1,5 \%$ minimum de la Surface de Vente | Opération $<400 \mathrm{~m}^{2}$ de Surface de Vente : pas d'obligation <br> Opération $\geq 400 \mathrm{~m}^{2}$ de Surface de Vente : Si réalisation de places de stationnement pour les voitures, alors 10 emplacements vélos +1 emplacement vélo/5 places de stationnement réalisées pour les voitures |
| Restauration / <br> Activités de services où s'effectue l'accueil d'une clientèle | Opération $<500 \mathrm{~m}^{2}$ de Surface de Plancher: pas d'obligation <br> Opération $\geq 500 \mathrm{~m}^{2}$ de Surface de Plancher: $1,5 \%$ minimum de la Surface de Vente | Opération $<500 \mathrm{~m}^{2}$ de Surface de Plancher: pas d'obligation <br> Opération $\geq 500 \mathrm{~m}^{2}$ de Surface de Plancher: Si réalisation de places de stationnement pour les voitures, alors 10 emplacements vélos +1 emplacement vélo/5 places de stationnement réalisées pour les voitures |
| Bureaux/ <br> Locaux et bureaux accueillant du public des administrations publiques et assimilés | 1 emplacement minimum pour $75 \mathrm{~m}^{2}$ de Surface de Plancher | 1 emplacement minimum pour $100 \mathrm{~m}^{2}$ de Surface de Plancher |
| Autres destinations et sous-destinations | Selon les besoins de l'opération |  |

# Normes de stationnement pour les vélos 

| Catégories <br> Destinations Sous-destinations Sous réserve qu'elles soient autorisées dans la zone! | Secteur S6 | Secteur SC |
| :--: | :--: | :--: |
| Logement | 2 emplacements jusqu'à $150 \mathrm{~m}^{2}$ de Surface de Plancher Au-delà, 2 emplacements pour les premiers $150 \mathrm{~m}^{2}$ de Surface de Plancher, puis 1 emplacement minimum pour $35 \mathrm{~m}^{2}$ de Surface de Plancher |  |
| Résidences universitaires | 1 emplacement minimum par place d'hébergement |  |
| Artisanat et commerce de détail | Opération <400m² de Surface de Vente; pas d'obligation <br> Opération $\geq 400 \mathrm{~m}^{2}$ de Surface de Vente: $1,5 \%$ minimum de la Surface de Vente | Opération $<400 \mathrm{~m}^{2}$ de Surface de Vente ; pas d'obligation <br> Opération $\geq 400 \mathrm{~m}^{2}$ de Surface de Vente : $1 \%$ minimum de la Surface de Vente |
| Restauration / <br> Activités de services où s'effectue l'accueil d'une clientèle | Opération <500m² de Surface de Plancher; pas d'obligation <br> Opération $\geq 500 \mathrm{~m}^{2}$ de Surface de Plancher: $1,5 \%$ minimum de la Surface de Vente | Opération <500m² de Surface de Plancher ; pas d'obligation <br> Opération $\geq 500 \mathrm{~m}^{2}$ de Surface de Plancher : $1 \%$ minimum de la Surface de Vente |
| Bureaux/ <br> Locaux et bureaux accueillant du public des administrations publiques et assimilés | 1 emplacement minimum pour $100 \mathrm{~m}^{2}$ de Surface de Plancher | 1 emplacement minimum pour $60 \mathrm{~m}^{2}$ de Surface de Plancher |
| Autres destinations et sous-destinations | Selon les besoins de l'opération |  |

### 7.2.3. Dispositions particulières

## - Constructions existantes, extensions, annexes et changements de destination

Dans le cas de travaux sur les constructions existantes, de création d'une annexe, d'extension ou de changement de destination, seule la surface de plancher créée est prise en compte pour calculer les obligations de réalisation d'aires de stationnement pour les cycles.

Cependant, pour les opérations créant moins de $60 \mathrm{~m}^{2}$ de surface de plancher, aucune place de stationnement n'est exigée.

# ARTICLE 8 - DESSERTE PAR LES VOIES PUBLIQUES ET PRIVÉES 

### 8.1. Accès

Les caractéristiques des accès doivent être définies en fonction de l'importance et de la destination des constructions et installations à réaliser, notamment en ce qui concerne la sécurité, la commodité de la circulation et l'approche des moyens de lutte contre l'incendie.

Le nombre d'accès doit être limité au strict nécessaire.
Les accès doivent être localisés et configurés en tenant compte :

- de la topographie et de la morphologie des lieux dans lesquels s'insère l'opération ;
- de la nature des voies sur lesquelles les accès sont susceptibles d'être aménagés afin de préserver la sécurité des personnes (visibilité, vitesse des véhicules, intensité du trafic);
- du type de trafic généré par l'opération (fréquence journalières, nombre de véhicules accédant au terrain, type de véhicules concernés) ;
- des conditions permettant l'entrée et la sortie des véhicules sur le terrain sans manœuvre sur la voie de desserte.

Lorsque le terrain est desservi par plusieurs voies, le projet doit privilégier la ou les solutions qui présentent la moindre gêne pour la circulation générale et permettent un accès aisé aux véhicules de secours et de services.

Lorsque le document graphique J «Plan des emplacements réservés» localise l'accès à un terrain depuis une voie ouverte au public par un point rouge, le projet de construction doit s'y conformer. Toute autre ouverture d'accès à ce terrain permettant la circulation automobile est prohibée.

### 8.2. Voiries

Pour toute nouvelle opération d'aménagement, la voirie interne doit présenter des caractéristiques adaptées à l'importance et à la destination des constructions à desservir. Elle doit être dimensionnée de manière à permettre une circulation aisée des véhicules de secours et de services à savoir :

- une largeur supérieure ou égale à 3 m ,
- une pente inférieure ou égale à $15 \%$,
- une hauteur libre supérieure ou égale à $3,50 \mathrm{~m}$.

Les voies en impasse doivent permettre le retournement des véhicules de secours et de service; si une voie en impasse de plus de 30 m est une voie-engin, il convient de porter sa largeur à 4 m .

Dans les périmètres couverts par des servitudes d'utilité publique pour la préservation des captages, figurant sur le document graphique B3 «Plan de prévention des pollutions»:

- Dans les périmètres de protection rapprochés des captages, la création ou l'augmentation de gabarit de voiries et de pistes autres que celles nécessaires au fonctionnement des destinations et usages autorisés ainsi que l'infiltration des eaux de ruissellement issues d'aires imperméabilisées sont interdits.

- Dans les périmètres de protection éloignés des captages, la création de nouvelles voiries et l'augmentation du gabarit des voiries existantes sont autorisées sous réserve que l'ensemble des dispositions techniques permettant de limiter l'impact sur les milieux naturels soient prises.


# ARTICLE 9 - DESSERTE PAR LES RÉSEAUX 

### 9.1. Alimentation en eau potable

Toute construction dont la destination implique qu'elle soit alimentée en eau potable, doit être approvisionnée soit par raccordement au réseau public d'eau potable, soit à partir de source, puits ou forage agréé. L'alimentation en eau potable doit être réalisée dans les conditions prévues par la réglementation en vigueur.

Dans les périmètres couverts par des servitudes d'utilité publique pour la préservation des captages, figurant sur le document graphique B3 «Plan de prévention des pollutions » et pour l'ensemble des usages et affectation des sols, le pétitionnaire doit démontrer que le projet n'aggrave pas la vulnérabilité des captages.

En présence de plusieurs règles, c'est la règlementation la plus contraignante qui doit être prise en compte.

### 9.2. Gestion des eaux usées domestiques

- Zones d'assainissement collectif définies au zonage d'assainissement (cf. document graphique B3 «Plan de prévention des pollutions »)

Le raccordement sur le réseau public d'assainissement est obligatoire pour toute construction ou installation générant des eaux usées. Ce raccordement doit respecter le règlement d'assainissement intercommunal de Grenoble-Alpes Métropole, annexé au PLU (Cf. annexe 2B2).

Seules les eaux usées peuvent être rejetées dans le réseau d'eaux usées.
Dans les périmètres de captages figurant sur le document graphique B3 «Plan de prévention des pollutions», les rejets d'eaux usées en milieu naturel sont interdits.

- Zones d'assainissement non-collectif définies au zonage d'assainissement (cf. document graphique B3 «Plan de prévention des pollutions »)

Les constructions doivent être équipées d'un système d'assainissement non collectif conforme à la réglementation en vigueur et adapté aux caractéristiques du sol, du sous-sol et de l'environnement du terrain. Toute infiltration concentrée est interdite dans les secteurs soumis à un aléa de glissement de terrain.

Dans les périmètres de protection rapprochés des captages, figurant sur le document graphique B3 «Plan de prévention des pollutions » tous les dispositifs d'assainissement non collectif comportant un puits d'infiltration sont interdits.

### 9.3. Gestion des eaux usées non domestiques

Les installations, ouvrages, travaux et activités réalisés à des fins non domestiques et entrainant des déversements, écoulements et rejets, même non polluants, doivent se conformer à la réglementation en vigueur.

Dans les périmètres de captages figurant sur le document graphique B3 «Plan de prévention des pollutions», les rejets d'eaux usées en milieu naturel sont interdits.

# 9.4. Utilisation du réseau d'eaux pluviales 

Les eaux pluviales doivent être gérées sur le terrain d'assiette du projet par tout dispositif approprié (noues, toitures végétalisées, tranchées infiltrantes etc.).

En cas d'impossibilité technique ou règlementaire avérée et justifiée ou au regard des enjeux de prévention des risques, toute adaptation de la règle sera conditionnée à l'abattement volumique des 15 premiers millimètres de pluie qui devront être gérés sur la parcelle sans débit de fuite.

Dans ce cas, des dispositifs de rétention sur place des eaux pluviales peuvent être admis au niveau de chaque parcelle et/ou de l'ensemble de l'opération, notamment quand la nature du sol le justifie. Dans ce cas, les eaux pluviales des parcelles sont stockées avant rejet à débit régulé (ratio maximum de $5 \mathrm{l} / \mathrm{ha} / \mathrm{s}$ ) dans le réseau d'eaux pluviales ou dans un exutoire naturel superficiel, sous réserve de leur existence, de leur capacité et de leur disponibilité. Les aménagements réalisés doivent garantir l'écoulement des seules eaux pluviales dans ce réseau, ainsi que, le cas échéant, celui des eaux assimilées à des eaux pluviales (eaux de refroidissement, eaux de piscines collectives etc.) avec la même régulation du rejet.

Les aménagements doivent permettre le libre écoulement des eaux pluviales sans aggraver la servitude de fonds inférieur.

Pour tout rejet à l'exutoire (naturel ou réseau), les raccordements avec stagnation d'eau à l'air libre sont interdits en dehors de dispositifs végétalisés.

Les aménagements extérieurs des constructions doivent contribuer à limiter l'imperméabilisation des sols.

Dans les périmètres de captages, identifiés sur le document graphique B3 «Plan de prévention des pollutions», les infiltrations superficielles et aériennes des eaux issues d'aires imperméabilisées ou de toitures sont autorisées à condition de ne pas aggraver la vulnérabilité des captages (infiltration directe interdite).

### 9.5. Réseaux électriques et téléphoniques

Le raccordement aux réseaux électriques et téléphoniques doit être réalisé soit par des câbles souterrains soit par toute autre technique permettant la dissimulation des câbles et ce jusqu'au réseau public qui existe au droit du terrain d'assiette du projet.

### 9.6. Déploiement de la fibre optique

Toute nouvelle opération d'aménagement doit intégrer la mise en place de gaines souterraines permettant le passage de la fibre optique dans des conditions permettant la desserte de l'ensemble des constructions projetées, y compris dans le cadre de la réalisation d'une opération d'aménagement d'ensemble.

Lorsque l'unité foncière est desservie par un ou plusieurs réseaux de communication électronique à très haut débit, toute construction nouvelle à usage d'habitation, de bureaux, de commerce, d'hôtels ou d'autres hébergements touristiques doit pouvoir y être raccordée. Afin de permettre le raccordement des nouvelles constructions, y compris

dans le cadre de la réalisation d'une opération d'aménagement d'ensemble, des fourreaux doivent être mis en attente en limite du domaine public (existant ou futur), en un point à déterminer en accord avec les services techniques des concessionnaires.

# ARTICLE 10 - ENERGIE ET PERFORMANCES ÉNERGÉTIQUES 

En complément des règles figurant ci-dessous, il convient de se reporter aux dispositions de l'articles 10 des règlements de zone.

## - Réseaux de chaleur

Les constructions nouvelles et l'ensemble des bâtiments existants situés dans les périmètres de développement prioritaire des réseaux de chaleur classés, doivent s'y raccorder selon les modalités et les cas prévus par les délibérations de classement des réseaux de chaleur (voir en annexe 4_A).

## - Constructions nouvelles

Les performances énergétiques des constructions nouvelles soumises à la règlementation thermique de 2012 doivent être renforcées de $20 \%$ par rapport à cette règlementation, en besoin climatique $\left(\mathrm{B}_{\mathrm{BO}}\right)$ et en consommation maximale en énergie primaire annuelle $\left(\mathrm{C}_{\mathrm{EP}}\right)$. La $\mathrm{C}_{\mathrm{EP}}$ doit être présentée brute, c'est-à-dire sans tenir compte de la production éventuelle d'énergie renouvelable.

Les auteurs des projets doivent tendre à intégrer les principes de l'architecture bioclimatique pour assurer le confort intérieur tant en hiver qu'en été. Ainsi, les constructions doivent être conçues (orientation/ dimensionnement/ protection des ouvertures) de manière à optimiser le rayonnement solaire en hiver pour favoriser les «apports passifs » et être protégées du soleil durant l'été par des dispositifs adaptés.

## - Production d'énergies renouvelables

Toute construction nouvelle soumise à la règlementation thermique et dont la surface de plancher est supérieure ou égale à $1000 \mathrm{~m}^{2}$ doit produire, qu'elle soit située ou non dans le périmètre de classement des réseaux de chaleur :

- au minimum $20 \mathrm{kWh}_{\mathrm{EF}} / \mathrm{m}^{2}$ d'emprise au sol / an, pour les immeubles à vocation dominante d'habitat, les commerces et les équipements publics (hors bureaux) et tout autre bâtiment soumis à la règlementation thermique ;
- au minimum $40 \mathrm{kWh}_{\mathrm{EF}} / \mathrm{m}^{2}$ d'emprise au sol / an, pour les immeubles à vocation dominante de bureaux, y compris des administrations.

![img-16.jpeg](img-16.jpeg)

Surface prise en compte pour le déclenchement de l'obligation (à partir de 1000 m² de SP)
Surface de plancher du res-de-chaussée de la construction Surface de plancher du premier niveau de la construction Surface de plancher du deuxième niveau de la construction Surface de plancher des combles aménagements
Surface sur laquelle s'applique le ratio de Heh renouvelable à produire (2) emprise au sol
(schéma illustratif)
Dans le cadre d'une opération d'aménagement d'ensemble, les différents modes de production pourront être regroupés sur certains bâtiments et/ou parkings au regard de leur localisation et/ou orientation.

L'ensemble des productions est calculé en énergie finale.

# - Rénovation/ réhabilitation 

Dans le cas de travaux relevant soit de la réglementation thermique sur l'existant, soit des obligations d'isolation rendues obligatoires par la loi de transition énergétique pour la croissance verte, des niveaux de performance énergétique sont exigés.
> En cas de travaux de ravalement et/ou de réfection de toiture et/ou d'aménagement de locaux en vue de les rendre habitables, pour lesquels des travaux d'isolation thermique sont obligatoires, le projet doit respecter les performances énergétiques définies pour les opérations standardisées du dispositif des Certificats d'économie d'énergie (CEE) correspondant aux travaux réalisés.

Les travaux concernés, ainsi que les cas dérogatoires (monuments historiques classés ou inscrits, bâti construits après 2001, etc.), sont définis par le décret $n^{\circ} 2016-711$ du 30 mai 2016, modifié par le décret $n^{\circ} 2017-919$ du 9 mai 2017.
> En cas de travaux impliquant une isolation par l'extérieur, le porteur projet doit justifier une animation des façades par un choix pertinent des matériaux et des éléments de décor. Lorsque l'isolation par l'extérieur n'est pas possible, des méthodes respectant le caractère architectural ou patrimonial (ex. : enduit isolant) doivent être privilégiées. En outre, le porteur de projet doit démontrer qu'il ne pénalise ni le confort d'été ni la qualité et le renouvellement de l'air intérieur du bâtiment.
> En cas de travaux de rénovation, d'installation ou de remplacement d'une paroi opaque ou vitrée d'un bâtiment existant, la performance de ce dernier doit respecter les caractéristiques thermiques définies pour les opérations standardisées du dispositif des Certificats d'économie d'énergie (CEE) correspondant aux travaux réalisés.

Les fiches sont disponibles sur le site :
https://www.ecologique-solidaire.gouv.fr/operations-standardisees-deconomies-denergie\#e4
> En cas de travaux sur des bâtiments postérieurs à 1948 de plus de $1000 \mathrm{~m}^{2}$ de SP rénovée et relevant de la règlementation thermique existante globale, le niveau de performance énergétique à atteindre est celui du label BBC Rénovation.

Pour les constructions situées dans le périmètre du site patrimonial remarquable (SPR) de Grenoble et les constructions protégées au titre du patrimoine et repérées sur le document graphique F2 "Plan du patrimoine bâti, paysager et écologique", les procédés destinés à l'amélioration du confort thermique avec enduit à caractère isolant (du type chaux chanvre etc.) sont autorisés dans la mesure où ils sont compatibles avec les supports existants et permettent la conservation de l'intégralité des décors et de la modénature de la façade.

![img-17.jpeg](img-17.jpeg)

Plan Local d'Urbanisme Intercommunal

# Règlement pièces écrites 

## Lexique

Dispositions générales - Tome 1.1
> PLUI approuvé le 20/12/2019 - Modification n¹ du 16/12/2022
![img-18.jpeg](img-18.jpeg)

LEXIQUE

# LEXIQUE 

LISTE DES MOTS DEFINIS DANS LE LEXIQUE :

LISTE DES MOTS DEFINIS DANS LE LEXIQUE :

| Acrotère | ICPE (Installation Classée pour la Protection de l'Environnement) |
| :--: | :--: |
| Activité commerciale de détail et de proximité | Infrastructure |
| Affouillement du sol |  |
| Aire de stationnement | Lambrequin |
| Aléa | Limite de fait |
| Aléa de référence | Limites séparatives |
| Alignement et limite de fait entre le terrain et la voie ou l'emprise publique | Linéaire commercial Local accessoire Local d'activité |
| Annexe | Logement social |
| Arbre de haute tige | Lucarne |
| Architecture bioclimatique | Lucarnes rampantes |
| Attique |  |
| Autopartage | Marge de recul |
|  | Matériaux biosourcés / Ecomatériaux |
| Balcon | Mélange terre-pierre |
| Bâtiment | Modénatures |
|  | Modes doux (ou actifs) de déplacement |
| Camping-caravaning, camping | Mur de soutènement |
| Centralité urbaine commerciale |  |
| Changement de destination | Niveau |
| Chantourne |  |
| Chiens assis | Opération d'aménagement d'ensemble |
| Clôture |  |
| Construction | Outeau |
| Construction existante |  |
| Contigu | Parking relais |
| Continu | Périmètre d'intensification urbaine |
| Corps principal | Persienne |
| Cours d'eau | Place de stationnement boxée |
| Couvertine | Pleine terre |
|  | Procédure loi sur l'eau |
| Débal |  |
| Débord de toiture | Regroupement commercial |
| Destinations des constructions | Rembilai, remblalement |
| Dispositions appropriées aux risques | Renouvellement urbain |
|  | Réseau hydrographique |
| Eaux pluviales | Ripisylve |
| Embâcle | Risque |
| Emprises publiques |  |
| Encorbellement | Serre-tunnel agricole |
| Energie finale | Sinistre |
| Energie grise | Skydome |
| Energie primaire | Surélévation |
| Energies renouvelables | Surface de plancher |
| Engrangeou | Surface de vente |
| Equipment sensible |  |
| Espace de développement commercial | Talweg |
| Espace végétalisé structurant | Terrain existant avant travaux (ou terrain naturel) |
| Etablissement commercial | Transparence hydraulique |
| Établissement de secours | Travaux et aménagements de nature à réduire les risques |
| Établissement recevant du public (ERP) | Tunnel agricole |
| Exhaussement du sol |  |
| Extension | Unité foncière |
| Extension mesurée |  |
| Exutoire naturel | Véhicule léger |
|  | Vélo cargo |
| Façade | Voie engin |
| Faitage | Voies ou emprises publiques |
| Fossé | Vulnérabilité (au titre des captages) / (au sens des risques) |
|  | Vulnérable |
| Gabarit |  |
| Génoise | ZAC |
|  | ARRETE DU 10 NOVEMBRE 2016 DEFINISSANT LES |
| Houppier d'un arbre | DESTINATIONS ET SOUS-DESTINATIONS DE CONSTRUCTIONS |

# Acrotère 

L'acrotère est l'élément d'une façade qui est situé au-dessus du niveau de la toiture ou de la terrasse et qui constitue des rebords ou des gardes corps pleins ou à claire voie.
![img-19.jpeg](img-19.jpeg)

## Activité commerciale de détail et de proximité

Est considérée comme activité de détail et de proximité toute activité commerciale de vente au détail ou artisanale, accompagnée de prestations de service située dans les pôles de vie, destinée aux particuliers et répondant :

- aux besoins quotidiens des consommateurs pour l'achat de produits et de services renouvelés fréquemment
- ou l'achat plus occasionnel de produits légers,


## A titre d'illustration, non exhaustives, sont concernées :

- les activités de vente de détail sur étalages et marchés ;
- les activités de vente de détail alimentaire en magasin spécialisé ou par automate ;
- les activités de vente de détail de textile ou d'habillement en magasin spécialisé ;
- les activités de vente de détail de chaussures et d'articles en cuir en magasin spécialisé ;
- les activités de vente de détail de produits pharmaceutiques en magasin spécialisé ou par automate ;
- les activités de vente de détail d'articles médicaux et orthopédiques en magasin spécialisé ;
- les activités de vente de détail de parfumerie et de produits de beauté en magasin spécialisé ou par automate ;
- les activités de vente de détail d'articles d'horlogerie et de bijouterie en magasin spécialisé ou par automate ;
- les activités de vente de détail de livres en magasin spécialisé ou par automate ;
- les activités de vente de détail de journaux et papeterie en magasin spécialisé ou par automate ;
- les activités de vente de détail ou de location d'enregistrements musicaux et vidéos en magasin spécialisé ou par automate ;
- les activités de vente de détail ou de location d'articles de sport en magasin spécialisé ;
- les activités de vente de détail de jeux et jouets en magasin spécialisé ;
- les activités de vente de détail d'optique en magasin spécialisé ;
- les activités de vente de détail de matériel photographique et de matériel de précision en magasin spécialisé ;
- les activités de vente de détail de souvenirs, d'objets artisanaux et d'articles religieux en magasin spécialisé ou par automate ;

- les activités des galeries d'art commerciales ainsi que les activités de vente de détail d'objets d'art, de timbres-poste ou de pièces de monnaie en magasin spécialisé ;
- les activités de vente de détail d'armes et de munitions en magasin spécialisé ;
- les activités de vente de détail en magasin non spécialisé, qu'elles soient ou non à prédominance alimentaire.


# Affouillement du sol 

Abaissement du niveau du sol.

## Aire de stationnement

Ensemble de places de stationnement de véhicules, matérialisées et comprenant plus de 2 places contigües. Un même projet peut comprendre plusieurs aires de stationnement. Exemple : un total de 20 places distribuées en 3 aires : une de 7 places, une de 3 places et une de 10 places.

## Aléa

Phénomène naturel (crue, chute de blocs, avalanche, etc.) ou technologique (surpression, toxique etc.) caractérisé en fréquence, en intensité et localisé.

## Aléa de référence

Aléa ayant servi de base à la réalisation d'un document définissant l'aléa (par exemple : carte d'aléa ou plan de prévention des risques naturels). Il correspond à une période de retour ou d'occurrence (par exemple: centennale). Il peut résulter de la combinaison de plusieurs scénarios.

## Alignement et limite de fait entre le terrain et la voie ou l'emprise publique

L'alignement ou la limite de fait entre le terrain assiette du projet et la voie ou l'emprise publique correspond à la limite actuelle ou future (emplacement réservé) entre la voie ouverte à la circulation publique et la propriété privée.

L'alignement peut résulter soit d'un état de fait, soit d'un plan d'alignement ou d'une mesure d'alignement individuelle.
![img-20.jpeg](img-20.jpeg)
(Schéma illustratif)

# Annexe 

Une annexe est une construction secondaire, de dimensions réduites et inférieures à celles de la construction principale, qui apporte un complément aux fonctionnalités de la construction principale. Elle doit être implantée selon un éloignement restreint par rapport à une autre construction, afin de marquer un lien d'usage. Elle peut être accolée ou non à la construction principale avec laquelle elle entretient un lien fonctionnel, sans disposer d'accès direct depuis la construction principale. En cas d'accès direct, il s'agit d'une extension.

Il est précisé :

- que l'annexe est nécessairement située sur la même unité foncière que la construction principale ;
- qu'elle a la même destination et sous-destination que la construction principale.
- qu'elle peut créer ou ne pas créer de surface de plancher au sens du code de l'urbanisme.
- qu'une piscine constitue une annexe (sauf mention contraire par le règlement).


## Arbre de haute tige

Arbre ayant une taille moyenne de 7 m à l'âge adulte.

## Architecture bioclimatique

On parle de conception bioclimatique lorsque l'architecture du projet est adaptée en fonction des caractéristiques et particularités du lieu d'implantation, afin d'en tirer le bénéfice des avantages et de se prémunir des désavantages et contraintes. L'objectif principal est d'obtenir le confort d'ambiance recherché de manière la plus naturelle possible en utilisant les moyens architecturaux, les énergies renouvelables disponibles et en utilisant le moins possible les moyens techniques mécanisés et les énergies extérieures au site. Ces stratégies et techniques architecturales cherchent à profiter au maximum du soleil en hiver et de s'en protéger durant l'été. C'est pour cela que l'on parle également d'architecture «solaire» ou «passive».

En règle générale, il est convenu que l'architecture bioclimatique propose :

- Une maximisation des surfaces vitrées orientées au Sud, protégés du soleil estival par des casquettes horizontales,
- Une minimisation des surfaces vitrées orientées au Nord. En effet, les apports solaires sont très faibles et un vitrage sera forcément plus déperditif qu'une paroi isolée,
- Des surfaces vitrées raisonnées et réfléchies pour les orientations Est et Ouest afin de se protéger des surchauffes estivales.

En hiver, une fois captée et transformée, l'énergie solaire doit être conservée à l'intérieur de la construction et valorisée au moment opportun. En été, c'est la fraicheur nocturne, captée via une sur-ventilation par exemple, qui doit être stockée dans le bâti afin de limiter les surchauffes pendant le jour. De manière générale, cette énergie est stockée dans les matériaux lourds de la construction. Afin de maximiser cette inertie, on privilégiera l'isolation par l'extérieur.

# Attique 

Est considéré comme attique le ou les derniers niveaux placés au sommet d'une construction et situés en retrait d'au moins 1 m des façades.
![img-21.jpeg](img-21.jpeg)

## Autopartage

Système qui permet à des abonnés de louer de façon ponctuelle une voiture en libre-service.

## Balcon

Plancher formant saillie sur une façade et ceint par une balustrade ou un garde-corps.

## Bâtiment

Un bâtiment est une construction couverte et close.
Un bâtiment constitue un sous-ensemble de la notion de construction. Il est réalisé en élévation et peut comprendre un sous-sol. II est impérativement couvert par une toiture et doté de systèmes de fermeture en permettant une clôture totale.

Ne peuvent donc être considérées comme relevant de la définition du bâtiment les constructions qui ne sont pas closes en raison :
-soit de l'absence totale ou partielle de façades closes ;
-soit de l'absence de toiture ;
-soit de l'absence d'une porte de nature à empêcher le passage ou la circulation (de type galerie).

## Camping-caravaning, camping

Notion couvrant les types d'aménagements suivants :

- les terrains de camping ou de camping-caravaning ;
- les parcs résidentiels de loisirs et les villages de vacances classés en hébergement léger ;
- l'installation d'une caravane en dehors d'un terrain de camping ;
- les résidences démontables occupées à titre de résidence principale définies par l'article R.111-51 du code de l'urbanisme ;
- les dépôts de véhicules et garages collectifs de caravanes ou de résidences mobiles de loisirs.

Les aires de grand passage sont exclues de la catégorie «camping-caravaning » lorsque le règlement édicte des règles spécifiques à leur égard ; elles sont incluses dans le cas contraire.

# Centralité urbaine commerciale 

En application de l'art L.151-16 du Code de l'urbanisme, il s'agit d'un secteur défini sur le document graphique C1 «Atlas de la mixité fonctionnelle et commerciale » dans lequel les activités commerciales de détail et de proximité sont autorisées.

## Changement de destination

Se reporter à «destinations des constructions».

## Chantourne

Fossé de drainage recevant fréquemment des cours d'eau secondaires issus des versants proches et conduisant leurs eaux vers les cours d'eau principaux (Isère notamment) parcourant les plaines ou vallées drainées. Terme local utilisé en Dauphiné. Provient du fait que les fossés de drainage «tournent autour des champs».

## Chiens assis

Un chien-assis est à l'origine une lucarne de petite dimension propre aux toits à faible pente, couverte par un rampant unique en pente inverse de celle du toit, destinée à assurer principalement une fonction de ventilation. Le chien-assis est l'équivalent d'une lucarne retroussée ou «demoiselle».
![img-22.jpeg](img-22.jpeg)

## Clôture

Toute enceinte qui ferme l'accès d'un terrain : mur, haie, grillage, palissade, portail, barrière.

## Construction

Une construction est un ouvrage fixe et pérenne, comportant ou non des fondations et générant un espace utilisable par l'Homme en sous-sol ou en surface.

La notion de construction recouvre notamment les constructions en surplomb (constructions sur pilotis, cabanes dans les arbres) et les constructions non comprises dans la définition du bâtiment, notamment les pergolas, hangars, abris de stationnement, piscines, les terrasses surélevées de 60 cm ou plus, les sous-sols non compris dans un bâtiment.

La notion d'espace utilisable par l'Homme vise à différencier les constructions des installations dans lesquelles l'Homme ne peut rentrer, vivre ou exercer une activité. Les constructions utilisées pour les exploitations agricoles, dans lesquelles l'Homme peut intervenir, entrent dans le champ de la définition. A contrario, les installations techniques de petites dimensions (chaufferie, éoliennes, poste de transformation, canalisations ...) et les murs et clôtures n'ont pas vocation à créer un espace utilisable par I'Homme.

# Construction existante 

Une construction est considérée comme existante si elle est reconnue comme légalement construite et si la majorité des fondations ou des éléments hors fondations déterminant la résistance et la rigidité de l'ouvrage remplissent leurs fonctions.

Ainsi une construction, édifiée soit avant la loi du 15 juin 1943 relative au permis de construire, soit conformément à l'autorisation délivrée à cet effet et devenue définitive, est considérée comme légalement construite (CE. 15 mars 2006, Ministre de l'équipement, req. $\mathrm{N}^{\circ} 266.238$ ).

Une ruine ne peut pas être considérée comme une construction existante.

## Contigu

Contigu signifie qui est attenant. Ce terme désigne quelque chose qui est mitoyen d'autre chose, qui le touche.

## Continu

Caractère de ce qui est ininterrompu.

## Corps principal

Partie du bâtiment ou un bâtiment lorsqu'il y en a plusieurs, qui comporte la porte d'entrée principale.

## Cours d'eau

Écoulement d'eaux courantes dans un lit naturel à l'origine, alimenté par une source. L'écoulement peut ne pas être permanent compte tenu des conditions hydrologiques et géologiques locales.

A noter que les drains qui composent le réseau hydrographique ne se limitent pas aux cours d'eau, la définition est bien plus large que celle de cours d'eau.

Cette définition diffère de celle de l'article L.215-7-1 du code de l'environnement dans le sens où l'écoulement peut avoir lieu de manière très occasionnelle : « Constitue un cours d'eau un écoulement d'eaux courantes dans un lit naturel à l'origine, alimenté par une source et présentant un débit suffisant la majeure partie de l'année. L'écoulement peut ne pas être permanent compte tenu des conditions hydrologiques et géologiques locales. »

## Couvertine

Elément de protection et d'étanchéité de la partie supérieure d'un muret ou d'un acrotère.

# Déblai 

Retrait de matériaux pour abaisser le sol ou supprimer un relief de celui-ci. Il a pour conséquence un affouillement du sol.

## Débord de toiture

Partie de la toiture en saillie par rapport au plan de la façade.

## Destinations des constructions

Les destinations de constructions sont :
$1^{\circ}$ Exploitation agricole et forestière ;
$2^{\circ}$ Habitation ;
$3^{\circ}$ Commerce et activités de service ;
$4^{\circ}$ Équipements d'intérêt collectif et services publics ;
$5^{\circ}$ Autres activités des secteurs secondaire ou tertiaire.
Article R.151-27 du code de l'urbanisme : Les destinations se décomposent en sous-destinations définies par l'article R.151-28.
L'arrêté du 10 novembre 2016 fixant la liste des destinations et des sous-destinations est reproduit en fin de lexique.
Le passage d'une destination ou d'une sous-destination à une autre constitue un changement de destination.
Les règles applicables aux changements de destination sont celles applicables aux nouvelles constructions, sauf mention contraire du règlement.

Dispositions appropriées aux risques
Ensemble de mesures prises par le maître d'ouvrage d'un projet permettant d'assurer :

- au sein du projet : la sécurité des personnes et l'absence de dommages aux biens ;
- pour les tiers : l'absence d'aggravation des risques pour les personnes et les biens


## Eaux pluviales

Eaux issues des eaux de pluie et de la fonte des neiges, de la grêle ou de la glace ainsi que les eaux d'infiltration. Ces eaux peuvent être utilisées ensuite pour les fontaines, l'arrosage, le lavage des jardins, des voies publiques ou privées et des cours d'immeubles ...

Sont aussi considérées comme eaux pluviales les eaux de vidange des piscines, les eaux de climatisation ou de pompes à chaleur (définition Régie assainissement Métro).

## Embâcle

Un embâcle est une accumulation naturelle de matériaux apportés par l'eau. Il peut s'agir d'accumulation de matériaux naturels (sédiments, graviers ou matériaux rocheux issus de l'érosion ou d'un glissement de terrain, branches ou troncs d'arbres, glace) ou d'origine humaine (par exemple : déchets de taille et de nature diverses, stocks de matériaux, mobilier urbain, véhicules).

# Emprises publiques 

Cette définition est prise pour application des articles 4 et 6 .
Les emprises publiques correspondent à des espaces ouverts au public qui ne relèvent pas de la notion de voie ; il s'agit par exemple des voies ferrées et tramways, des cours d'eau domaniaux, des canaux, des jardins et parcs publics, des places publiques. Pour un complément d'information, se référer à la définition de "voies et emprises publiques".

## Encorbellement

Il qualifie l'étage d'une bâtisse qui s'avance sur une rue ou sur une place, de sorte que le rez-de-chaussée a une surface inférieure à celle de l'étage, qui lui-même a une surface moins grande que celle de l'étage supérieur et ainsi de suite.

## Energie finale

Il s'agit de l'énergie mise à disposition de l'utilisateur, celle livrée et facturée à chaque bâtiment.

## Energie grise

L'énergie grise correspond à la dépense énergétique totale pour l'élaboration d'un matériau, tout au long de son cycle de vie, de son extraction à son recyclage en passant par sa transformation, une énergie évaluée en $\mathrm{kWh} /$ tonne.

## Energie primaire

Il s'agit de l'énergie disponible à la source avant toute transformation. C'est l'unité à respecter dans l'étude thermique à joindre à la demande de permis de construire, avec des coefficients de conversion différents selon les énergies.

## Energies renouvelables

Les énergies renouvelables sont définies à l'article L.211-2 du Code de l'Energie. Il s'agit notamment de l'énergie hydraulique, de l'énergie solaire photovoltaïque, de l'énergie solaire thermique, de la géothermie, du bois énergie, du biogaz, etc. Les combustibles fossiles ou fissiles et la récupération d'énergie fatale ne sont pas des sources d'énergies renouvelables. Le règlement parle de production d'énergie renouvelable, c'est-à-dire d'une production sur le site à partir d'une ressource renouvelable.

## Engrangeou

Architecture vernaculaire : forme caractéristique de Mens. Grande ouverture dans le plan de la façade avec un large débord permettant de suspendre une poulie pour hisser le foin dans les combles.

## Equipement sensible

Les établissements sensibles, c'est-à-dire les bâtiments publics ou privés abritant une population sensible, ou hébergeant des populations vulnérables physiquement, sont définis par l'OAP Qualité de l'Air ; il s'agit des équipements suivants :

- Petite enfance : crèche, halte-garderie, maison de l'enfance,
- Etablissements scolaires : écoles primaires, collèges, lycées,

- Etablissements hospitaliers : cliniques et hôpitaux,
- Etablissements pour personnes âgées : maisons de retraite, foyer, MAPA(D),
- Etablissements sociaux : IME/IMA, maison pour tous, MJC...
- Équipements sportifs extérieur : stade...


# Espace de développement commercial 

Secteur défini sur le document graphique C1 «Atlas de la mixité fonctionnelle et commerciale» pouvant accueillir les constructions dédiées à l'artisanat et au commerce de détail et au commerce de gros.

## Espace végétalisé structurant

Espace composé d'éléments végétalisés qui organisent, articulent, ordonnent cet espace selon un plan de manière à lui donner un aspect défini et précis et à améliorer la qualité du paysage urbain.

## Etablissement commercial

L'établissement commercial est entendu comme un lieu d'exploitation commerciale, et donc rattaché à un fonds de commerce ou à une activité, contrairement au siège social.

Commerce isolé disposant d'une entrée sur un espace extérieur. Voir par ailleurs la définition de "regroupement commercial".

## Établissement de secours

Établissement intervenant dans la gestion de crise en cas de survenue des risques naturels (sapeurs-pompiers, gendarmerie, central téléphonique, centres de secours, de soins, héliport...).

## Établissement recevant du public (ERP)

Constituent des établissements recevant du public tous les bâtiments, locaux et enceintes dans lesquels des personnes sont admises, soit librement, soit moyennant une rétribution ou une participation quelconque, ou dans lesquels sont tenues des réunions ouvertes à tout venant ou sur invitation, payantes ou non.

La classification des ERP en types, groupes et catégories fait l'objet d'une description dans le glossaire du règlement type PPRn dans leTome 1.2 du règlement, à la définition de Classe de vulnérabilité des constructions.

## Exhaussement du sol

Augmentation du niveau du sol, surélévation du sol.

## Extension

L'extension consiste en un agrandissement de la construction existante présentant des dimensions inférieures à celle-ci. L'extension peut être horizontale ou verticale (par surélévation, excavation ou agrandissement) et doit présenter un lien physique et fonctionnel avec la construction existante.

L'élément essentiel caractérisant l'extension est sa contiguïté avec la construction principale existante. Sont considérées comme contiguës les constructions accolées l'une avec l'autre.

L'extension doit également constituer un ensemble architectural avec la construction principale existante. Le lien physique et fonctionnel doit être assuré soit par une porte de communication entre la construction existante et son extension, soit par un lien physique (par exemple dans le cas d'une piscine ou d'une terrasse prolongeant le bâtiment principal).

Une extension ne constitue pas une construction nouvelle.
![img-23.jpeg](img-23.jpeg)

# Extension mesurée 

Une extension mesurée est limitée à la création de $30 \%$ de surface de plancher supplémentaire par rapport à la surface de plancher existante avant travaux.

Exutoire naturel (pour la gestion des eaux usées traitées et des eaux de pluie)
Il s'agit du point bas d'un réseau hydrographique ou d'un milieu hydraulique superficiel, ce dernier devant être un cours d'eau permanent ou intermittent et non un simple talweg.

## Façade

Les façades d'un bâtiment ou d'une construction correspondent à l'ensemble de ses parois extérieures hors toiture. Elles intègrent tous les éléments structurels, tels que les baies, les bardages, les ouvertures, l'isolation extérieure et les éléments de modénature.

## Faltage

Ligne de jonction supérieure entre deux pans de toiture inclinés suivant des pentes opposées, par conséquent la partie la plus élevée d'un toit.

## Fossé

Il s'agit d'un aménagement à ciel ouvert ayant pour rôle de faciliter l'écoulement des eaux pouvant rejoindre directement ou indirectement un cours d'eau. Il n'est pas nécessairement en eaux permanentes.

## Gabarit

Le gabarit désigne l'ensemble des plans verticaux, horizontaux ou obliques constituant la forme extérieure de la construction. Il résulte de la combinaison des règles de hauteur, de prospects et d'emprise au sol.

# Génoise 

La génoise est une fermeture d'avant-toit (la face inférieure d'un avant-toit) formée de plusieurs rangs (de un à quatre, rarement plus) de tuiles-canal en encorbellement sur le mur. Le rôle de la génoise est d'une part d'éloigner les eaux de ruissellement de la façade comme une corniche, et d'autre part de supporter et continuer le pan versant de toit.
![img-24.jpeg](img-24.jpeg)

## Houpper d'un arbre

Synonyme de couronne d'un arbre, il s'agit de la partie constituée de l'ensemble des branches situées au sommet du tronc (des branches maîtresses aux rameaux).

ICPE (Installation Classée pour la Protection de l'Environnement)
Établissement relevant de la nomenclature des installations classées pour la protection de l'environnement.

## Infrastructure

Ensemble de constructions, d'ouvrages et d'aménagements permanents qui conditionnent le fonctionnement de l'activité économique d'une région, d'un pays.

Utilisé aussi dans le règlement pour désigner les parties communes d'un aménagement (voies, ponts...).

## Lambrequin

Ornement découpé et souvent ajouré, plus ou moins épais, en bois ou en métal (zinc, fer-blanc ou fonte de fer), fixé en bordure de toit ou à la partie supérieure d'une fenêtre.

## Limite de fait

Limite constatée entre le terrain d'assiette du projet et la voie ou l'emprise publique.
Se reporter à la définition de Voies et emprises publiques

(Schéma illustratif)
![img-25.jpeg](img-25.jpeg)

# Limites séparatives 

Les limites séparatives correspondent aux limites entre le terrain d'assiette de la construction, constitué d'une ou plusieurs unités foncières, et le ou les terrains contigus. Elles peuvent être distinguées en deux types : les limites latérales et les limites de fond de terrain. En sont exclues les limites de l'unité foncière par rapport aux voies et emprises publiques.

A la différence de la limite latérale, la limite de fond de parcelle n'a aucun contact avec une voie ou emprise publique.
![img-26.jpeg](img-26.jpeg)

## Linéaire commercial

En application de l'article L.151-16 du Code de l'urbanisme, les linéaires commerciaux figurant sur le document graphique C1 «Atlas de la mixité fonctionnelle et commerciale »indiquent précisément là où les changements de destination des cellules commerciales sont limités ou interdits. L'identification de ces linéaires est précise, pour avoir une continuité de cellules commerciales constituant une «masse critique» et présentant une «continuité » de cellules commerciales (ex : au moins 3). Ils peuvent être utilisés pour un alignement existant ou en projet.

## Local accessoire

Les locaux accessoires dépendent ou font partie intégrante d'une construction principale à laquelle ils apportent une fonction complémentaire et indissociable.

Il peut s'agir soit d'une annexe, soit d'une extension.
Ils peuvent recouvrir des constructions de nature très variée et être affectés à des usages divers : garage d'une habitation ou d'un bureau, atelier de réparation, entrepôt d'un commerce, remise, logement pour le personnel, lieu de vie du gardien d'un bâtiment industriel, local de stockage pour un commerce, laverie d'une résidence étudiante ...

De plus, conformément à l'article R.151-29 du code de l'urbanisme, les locaux accessoires sont réputés avoir la même destination et sous-destination que le local principal auquel ils se rattachent.

# Local d'activité 

Local de sous-destination autre que logement, hébergement, hôtel, autre hébergement touristique.

## Logement social

Sont comptabilisés dans le pourcentage de logements sociaux :

- Les logements et hébergements locatifs sociaux financés par des prêts type :
- PLAI : prêt locatif aidé,
- PLUS : prêt locatif à usage social,
- PLS : prêt locatif social
- Les logements en accession sociale via le Bail Réel Solidaire (BRS)


## Lucarne

Petite fenêtre de forme variée faisant généralement saillie, pratiquée dans le toit d'un bâtiment pour donner du jour et de l'air aux combles.

## Lucarnes rampantes

La lucarne rampante ou chien-couché est une lucarne de toiture à versant unique en pente légère mais de même sens que celle du toit principal (à la différence du chien-assis).
![img-27.jpeg](img-27.jpeg)

## Marge de recul

Recul minimum imposé le long des voies et emprises publiques. Il se compte à partir de la limite de fait, de l'alignement actuel ou de l'élargissement futur (emplacement réservé). Cette marge de recul peut être règlementée par l'article 4.1 du règlement écrit ou par le document graphique D1 «Atlas des formes urbaines : implantations et emprises».

## Matériaux biosourcés / Ecomatériaux

Les matériaux biosourcés sont des matériaux fabriqués à l'aide de matières premières naturelles d'origine végétale ou animale. Ils trouvent de multiples applications dans le domaine du bâtiment et de la construction comme par exemples les isolants (laines de fibres végétales ou animales, de liège, ouate de cellulose, etc.), les mortiers et bétons (béton de bois, de lin, etc.) et les panneaux (particules ou fibres végétales, paille compressée, etc.). On peut également les définir comme des matériaux de construction qui

répondent aux critères techniques habituellement exigés des matériaux de construction (performances techniques et fonctionnelles, qualité architecturale, durabilité, etc.), mais aussi à des critères environnementaux ou socio-environnementaux, tout au long de son cycle de vie (c'est-à-dire de sa production à son élimination ou recyclage).

# Mélange terre-pierres: 

Il permet la végétalisation d'espaces en offrant une portance au sol afin qu'il puisse accueillir ponctuellement une circulation de véhicules (lieux peu circulés ou circulés périodiquement) ou d'autres usages. Lorsqu'il n'est pas recouvert de végétation et qu'il est à proximité immédiate de plantations, il offre des espaces pour «stocker l'air et l'eau» et permet au système racinaire de s'étendre.

Le mélange terre-pierres est composé de :

- $60 \%$ de pierres. La taille des granulats est primordiale : comprise entre 20 mm et 40 mm (grave 20/40). Leur nature est secondaire.
- $40 \%$ de terre végétale.
![img-28.jpeg](img-28.jpeg)


## Modénatures

Ensemble des différents décors de la façade et notamment les encadrements, corniches, bandeaux ...

## Modes doux (ou actifs) de déplacement

Les modes doux (ou actifs) rassemblent les modes de déplacement de proximité non motorisés tels que la marche à pied, le vélo, la trottinette ou les rollers. Par extension, les modes de déplacement similaires, électriques ou à assistance électrique (VAE) sont compris dans cette catégorie (VAE, trottinettes électriques, gyropodes, hoverboards, segways...). Les transports en commun ne sont pas compris dans les modes doux de déplacements.

## Mur de soutènement

Ouvrage de maçonnerie destiné à soutenir, contenir, s'opposer aux poussées du terrain existant avant travaux.

## Niveau

Les niveaux sont mesurés de la même façon que la hauteur à la verticale de chaque point de la construction par rapport au terrain naturel.

![img-29.jpeg](img-29.jpeg)

Ne sont pas comptés en tant que niveaux :

- les parties de locaux et autres dépendances des constructions d'une hauteur inférieure à $1,80 \mathrm{~m}$ sous plafond ;
- les combles non aménageables sans fenêtre,
- les locaux en sous-sols semi enterrés dans la limite d'une émergence d'un mètre au-dessus du terrain naturel.
![img-30.jpeg](img-30.jpeg)
(Schéma illustratif)
![img-31.jpeg](img-31.jpeg)

# Opération d'aménagement d'ensemble 

Opération d'aménagement de type ZAC, lotissement, permis d'aménager ou permis valant division.

# Outeau 

Petite ouverture plate, triangulaire ou ronde, fermée par une grille et insérée dans une toiture en ardoise ou en tuiles plates ajustées pour effectuer la ventilation des combles.
![img-32.jpeg](img-32.jpeg)

## Parking relais

Les parcs relais, parkings relais, $\mathrm{P}+\mathrm{R}$ ou stationnements incitatifs sont des espaces de stationnement, situés en périphérie d'une ville, destinés à inciter les usagers à accéder au centre-ville en transport doux, en leur permettant de remiser leur véhicule dans un parking jusqu'à leur retour.

## Périmètre d'intensification urbaine

Périmètre au sein desquels le SCOT (Schéma de Cohérence territoriale) impose des densités minimales à respecter pour les nouvelles opérations d'habitation, notamment autour des arrêts de transports en commun.

## Persienne

Contrevent fermant une baie, composé d'une seule pièce ou de plusieurs vantaux et comportant (à la différence du volet, qui est plein et intérieur) un assemblage à claire-voie de lamelles inclinées qui arrêtent les rayons directs du soleil tout en laissant l'air circuler.

## Place de stationnement boxée

Une place boxée est une place de stationnement close individuellement par des murs et une porte.

## Pleine terre

Sont pris en compte pour la quantification des espaces de pleine terre :

- Les espaces de terre végétale, (dont l'épaisseur sera adaptée à la typologie des plantations), libres non couverts, non bâtis ni en surface ni en sous-sol, permettant la libre infiltration des eaux pluviales et aménagés en espaces verts (pelouses, plantations).
- Les espaces de terre végétale situés au-dessus des canalisations et des bassins de d'infiltration réalisés sans ouvrages. .

Ne sont pas comptabilisés en espaces de pleine terre :

- Les surfaces situées au pied des arbres isolés, imposés sur les aires de stationnement et inférieures aux diamètres de fosses de plantation décrits dans l'OAP Paysage et Biodiversité, diamètres qui varient selon la taille des arbres et l'ambiance.
- les espaces de circulation et de manœuvre des véhicules, notamment les espaces enherbés situés entre les roues des véhicules ;
- Les espaces de terre situés sur des bassins de rétention étanches ;
- Les mélanges terre-pierre.

# Procédure loi sur l'eau 

Procédure de déclaration ou d'autorisation en application des articles L. 214-1 à L. 214-6 du code de l'environnement, préalable à la mise en œuvre des installations, ouvrages, travaux et activités ayant une influence notable sur l'eau ou le fonctionnement des écosystèmes aquatiques. Les installations, ouvrages, travaux et activités concernés sont définis par l'article R. 214-1 du même code.

## Regroupement commercial

Ensemble de commerces regroupés dans un même bâtiment et reliés entre eux par des espaces de circulation intérieure. Lorsque des établissements commerciaux disposent à la fois d'une entrée indépendante et d'un accès par un espace de circulation intérieur, ils font partie d'un regroupement commercial.

## Remblai, remblaiement

Apport de matériaux pour élever un terrain ou combler un creux. Il a pour conséquence un exhaussement du sol. Également utilisé par le règlement au sens de zone surélevée par rapport au sol environnant.

## Renouvellement urbain

Forme d'évolution de la ville qui désigne l'action de reconstruction de la ville sur elle-même et de recyclage de ses ressources bâties et foncières.

## Réseau hydrographique

Ensemble des éléments naturels (rivières) ou artificiels (réseau), drainant un bassin versant.

## Ripisylve

Formations végétales boisées, buissonnantes et herbacées, qui se développent sur les berges des cours d'eau ou des plans d'eau. Elles sont constituées de boisements particuliers en raison de la présence d'eau sur des périodes plus ou moins longues : saules, aulnes, frênes en bordure, érables et ormes en hauteur, chênes pédonculés et charmes sur le haut des berges.

## Risque

Dommage potentiel aux enjeux (population, biens, activités) consécutif à la survenance d'un aléa.

Le risque est fonction de l'importance :

- de l'aléa ;
- des enjeux ;
- de la vulnérabilité des enjeux à l'aléa.

# Serre-tunnel agricole 

Structure démontable, en forme de demi-cercle dont la couverture est assurée par un film imperméable. Il ne dispose pas de fondation ni de dalle. Il revêt des usages agricoles permettant entre autres le maraîchage.

## Sinistre

Cette définition est établie pour application de l'article L.111-15 du Code de l'Urbanisme relatif à la reconstruction à l'identique d'un bâtiment sinistré.

Un sinistre est un évènement fortuit qui peut être accidentel (incendie, explosion...), dû à un acte de malveillance (attentat), ou la conséquence d'une catastrophe naturelle.

## Skydome

Sorte de hublot de plafond, lanterneau vitré pour éclairage zénithal.

## Surélévation

Se reporter à la définition de l'extension.

## Surface de plancher

Se reporter à la définition en vigueur dans le Code de l'Urbanisme.

## Surface de vente

La surface de vente d'un magasin de commerce de détail comprend la superficie des espaces couverts et non couverts, affectés :

- à la circulation de la clientèle pour effectuer ses achats ;
- à l'exposition des marchandises proposées à la vente ;
- au paiement des marchandises ;
- à la circulation du personnel pour présenter les marchandises à la vente.

En sont notamment exclus:

- les mails des centres commerciaux desservant plusieurs commerces sous réserve que n'y soit exposée aucune marchandise destinée à la vente ;
- les sas d'entrée des magasins, s'ils n'accueillent pas de marchandises proposées à la vente ;
- les ateliers d'entretien, de réparation, de fabrication ou de préparation des marchandises proposées à la vente, si leur accès est interdit au public.


## Talweg

Ligne représentant le fond d'une vallée (sèche ou en eau), elle relie les points les plus bas de cette vallée.

# Terrain existant avant travaux (ou terrain naturel) 

Terrain existant à la date de la demande d'autorisation d'urbanisme.
Le niveau du sol précité peut intégrer les modifications de niveau intervenues avant le dépôt de la demande, lesquelles ne doivent pas avoir de lien avec les travaux envisagés.

En outre, ces aménagements ne doivent pas avoir été réalisés dans un objectif frauduleux visant à fausser l'appréciation de l'administration sur la conformité de la construction projetée à la réglementation d'urbanisme applicable.

## Transparence hydraulique

Aptitude que possède un ouvrage ou un aménagement à ne pas faire obstacle aux mouvements des eaux. Globalement, un ouvrage est dit "transparent" d'un point de vue hydraulique lorsqu'il n'amplifie pas le niveau des plus hautes eaux, ne réduit pas la zone d'expansion des crues, n'allonge pas la durée ou l'étendue des inondations, et n'intensifie pas la vitesse d'écoulement des eaux...

Une construction transparente hydrauliquement doit donc laisser passer très largement l'eau.

## Travaux et aménagements de nature à réduire les risques

Les risques visés sont les risques naturels.
Les travaux et aménagements concernés sont ceux pour lesquels la réduction des risques est un des objectifs principaux. Ils peuvent être de nature collective (par exemple : protection des berges d'un cours d'eau par des enrochements) ou individuelle (par exemple : création d'une zone refuge).

## Tunnel agricole

Structure démontable, en forme de demi-cercle dont la couverture est assurée par une bâche imperméable. Il ne dispose pas de fondation ni de dalle. Il revêt des usages agricoles permettant le stockage (foin, matériel), ou l'élevage.

## Unité foncière

La notion d'unité foncière, ou tènement, a été définie par le Conseil d'Etat comme un «îlot de propriété d'un seul tenant composé d'une parcelle ou d'un ensemble de parcelles appartenant à un même propriétaire ou à la même indivision » (CE, 27 juin 2005, commune Chambéry c/ Balmat, $\mathrm{n}^{\circ} 264667$ ).

Un ensemble de terrains, de parcelles cadastrales ou lots réunis entre les mains d'un même propriétaire ne constitue une unité foncière que si et seulement si elle représente un ensemble d'un seul tenant dont la continuité foncière n'est pas interrompue.

## Véhicule léger

Catégorie de véhicules qui regroupe les voitures de tourisme et les véhicules utilitaires jusqu'à 3,5 tonnes.

## Vélo cargo

Un vélo à deux ou trois roues, spécifiquement conçu pour le transport de fret volumineux ou de personnes (par exemple des enfants). Il utilise pour cela un équipement spécifique inamovible.

# Voie engin 

Une voie d'accès au bâtiment, aménagée pour les véhicules de secours et de lutte contre l'incendie.

## Voies ou emprises publiques

Cette définition est prise pour application des articles 4 et 6 .
La voie publique s'entend comme la voie ouverte à la circulation publique.
Les voies ouvertes à la circulation publique recouvrent toutes les voies publiques ou voies privées ouvertes à la circulation publique, y compris les futures voies des lotissements, quel que soit leur statut ou leur fonction : circulation des véhicules motorisés, les itinéraires cyclables, l'emprise réservée au passage des piétons.

Les emprises publiques correspondent à des espaces ouverts au public qui ne relèvent pas de la notion de voie ; il s'agit par exemple des voies ferrées et tramways, des cours d'eau domaniaux, des canaux, des jardins et parcs publics, des places publiques.

## Vulnérabilité (au titre des captages)

La vulnérabilité d'un captage est liée à la fois à un certain nombre de caractéristiques intrinsèques de l'aire d'alimentation du captage (géologie, couverture pédologique, pentes...) et aux pressions exercée sur ce territoire (ensemble des activités susceptibles de générer des pollutions chroniques ou accidentelles des masses d'eau qui alimentent le captage).

## Vulnérabilité (au sens des risques)

Niveau de conséquences prévisibles sur un enjeu du phénomène de référence pris en compte dans le cadre de la réglementation au titre de la prévention des risques.

## Vulnérable

Elément susceptible de subir des dommages en cas de survenue du phénomène de référence pris en compte dans le cadre de la réglementation au titre de la prévention des risques.

## ZAC

Zone d'Aménagement Concerté : «Les zones d'aménagement concerté sont les zones à l'intérieur desquelles une collectivité publique ou un établissement public y ayant vocation décide d'intervenir pour réaliser ou faire réaliser l'aménagement et l'équipement des terrains, notamment de ceux que cette collectivité ou cet établissement a acquis ou acquerra en vue de les céder ou de les concéder ultérieurement à des utilisateurs publics ou privés. » (extrait article L. 311-1 code urbanisme).

Se référer aux textes en vigueur relatifs aux ZAC dans le Code de l'Urbanisme.

ARRETE DU 10 NOVEMBRE 2016
DEFINISSANT LES DESTINATIONS ET SOUS-DESTINATIONS DE CONSTRUCTIONS
POUVANT ETRE REGLEMENTEES PAR LE REGLEMENT NATIONAL D'URBANISME
ET LES REGLEMENTS DES PLANS LOCAUX D'URBANISME
OU LES DOCUMENTS EN TENANT LIEU

L'arrêté définit les sous-destinations de constructions pouvant être réglementées par les plans locaux d'urbanisme en application des articles R. 151-27, R. 151-28 et R. 151-29 du code de l'urbanisme.

Article 1 : La destination de construction « exploitation agricole et forestière » prévue au $1^{\circ}$ de l'article R. 151-27 du code de l'urbanisme comprend les deux sous-destinations suivantes : exploitation agricole, exploitation forestière.

La sous-destination « exploitation agricole » recouvre les constructions destinées à l'exercice d'une activité agricole ou pastorale. Cette sous-destination recouvre notamment les constructions destinées au logement du matériel, des animaux et des récoltes.

La sous-destination « exploitation forestière » recouvre les constructions et les entrepôts notamment de stockage du bois, des véhicules et des machines permettant l'exploitation forestière.

Article 2 : La destination de construction « habitation » prévue au $2^{\circ}$ de l'article R. 151-27 du code de l'urbanisme comprend les deux sous-destinations suivantes : logement, hébergement.

La sous-destination « logement » recouvre les constructions destinées au logement principal, secondaire ou occasionnel des ménages à l'exclusion des hébergements couverts par la sous-destination «hébergement». La sous-destination « logement » recouvre notamment les maisons individuelles et les immeubles collectifs.

La sous-destination « hébergement » recouvre les constructions destinées à l'hébergement dans des résidences ou foyers avec service. Cette sous-destination recouvre notamment les maisons de retraite, les résidences universitaires, les foyers de travailleurs et les résidences autonomie.

Article 3 : La destination de construction « commerce et activité de service » prévue au $3^{\circ}$ de l'article R. 151-27 du code de l'urbanisme comprend les six sous-destinations suivantes : artisanat et commerce de détail, restauration, commerce de gros, activités de services où s'effectue l'accueil d'une clientèle, cinéma, hôtels, autres hébergements touristiques.

La sous-destination « artisanat et commerce de détail » recouvre les constructions commerciales destinées à la présentation et vente de bien directe à une clientèle ainsi que les constructions artisanales destinées principalement à la vente de biens ou services.

La sous-destination « restauration » recouvre les constructions destinées à la restauration ouverte à la vente directe pour une clientèle commerciale.

La sous-destination « commerce de gros » recouvre les constructions destinées à la présentation et la vente de biens pour une clientèle professionnelle.

La sous-destination « activité de service où s'effectue l'accueil d'une clientèle » recouvre les constructions destinées à l'accueil d'une clientèle pour la conclusion directe de contrat de vente de services ou de prestation de services et accessoirement la présentation de biens.

La sous-destination « cinéma » recouvre toute construction répondant à la définition d'établissement de spectacles cinématographiques mentionnée à l'article L. 212-1 du code du cinéma et de l'image animée accueillant une clientèle commerciale.

La sous-destination « hôtels» recouvre les constructions destinées à l'accueil de touristes dans des hôtels, c'est-à-dire des établissements commerciaux qui offrent à une clientèle de passage qui, sauf exception, n'y élit pas domicile, des chambres ou des appartements meublés en location, ainsi qu'un certain nombre de services.

La sous-destination « autres hébergements touristiques » recouvre les constructions autres que les hôtels destinées à accueillir des touristes, notamment les résidences de tourisme et les villages de vacances, ainsi que les constructions dans les terrains de camping et dans les parcs résidentiels de loisirs.

Article 4 : La destination de construction « équipements d'intérêt collectif et services publics » prévue au $4^{\circ}$ de l'article R. 151-27 du code de l'urbanisme comprend les six sous-destinations suivantes : locaux et bureaux accueillant du public des administrations publiques et assimilés, locaux techniques et industriels des administrations publiques et assimilés, établissements d'enseignement, de santé et d'action sociale, salles d'art et de spectacles, équipements sportifs, autres équipements recevant du public.

La sous-destination « locaux et bureaux accueillant du public des administrations publiques et assimilés » recouvre les constructions destinées à assurer une mission de service public. Ces constructions peuvent être fermées au public ou ne prévoir qu'un accueil limité du public. Cette sous-destination comprend notamment les constructions de l'Etat, des collectivités territoriales, de leurs groupements ainsi que les constructions des autres personnes morales investies d'une mission de service public.

La sous-destination « locaux techniques et industriels des administrations publiques et assimilés » recouvre les constructions des équipements collectifs de nature technique ou industrielle. Cette sous-destination comprend notamment les constructions techniques nécessaires au fonctionnement des services publics, les constructions techniques conçues spécialement pour le fonctionnement de réseaux ou de services urbains, les constructions industrielles concourant à la production d'énergie.

La sous-destination « établissements d'enseignement, de santé et d'action sociale » recouvre les équipements d'intérêts collectifs destinés à l'enseignement ainsi que les établissements destinés à la petite enfance, les équipements d'intérêts collectifs hospitaliers, les équipements collectifs accueillant des services sociaux, d'assistance, d'orientation et autres services similaires.

La sous-destination « salles d'art et de spectacles » recouvre les constructions destinées aux activités créatives, artistiques et de spectacle, musées et autres activités culturelles d'intérêt collectif.

La sous-destination « équipements sportifs » recouvre les équipements d'intérêts collectifs destinées à l'exercice d'une activité sportive. Cette sous-destination comprend notamment les stades, les gymnases ainsi que les piscines ouvertes au public.

La sous-destination « autres équipements recevant du public » recouvre les équipements collectifs destinées à accueillir du public afin de satisfaire un besoin collectif ne répondant à aucune autre sous-destination définie au sein de la destination « Equipment d'intérêt collectif et services publics ». Cette sous-destination recouvre notamment les lieux de culte, les salles polyvalentes, les aires d'accueil des gens du voyage.

Article 5 : La destination de construction « autres activités des secteurs secondaire ou tertiaire » prévue au $5^{\circ}$ de l'article R. 151-27 du code de l'urbanisme comprend les quatre sous-destinations suivantes : industrie, entrepôt, bureau, centre de congrès et d'exposition.

La sous-destination «industrie» recouvre les constructions destinées à l'activité extractive et manufacturière du secteur primaire, les constructions destinées à l'activité industrielle du secteur secondaire ainsi que les constructions artisanales du secteur de la construction ou de l'industrie. Cette sous-destination recouvre notamment les activités de production, de construction ou de réparation susceptibles de générer des nuisances.

La sous-destination « entrepôt » recouvre les constructions destinées au stockage des biens ou à la logistique.
La sous-destination «bureau» recouvre les constructions destinées aux activités de direction et de gestion des entreprises des secteurs primaires, secondaires et tertiaires.

La sous-destination « centre de congrès et d'exposition » recouvre les constructions destinées à l'événementiel polyvalent, l'organisation de salons et forums à titre payant.

![img-33.jpeg](img-33.jpeg)

# GRENOBLEALPES MÉTROPOLE 

## L'AGENCE

D'URBANISME DE LA RÉGION GREENOBLOISE

## FLUI

Plan Local d'Urbanisme Intercommunal Grenoble-Alpes Métropole

GRENOBLE-ALPES MÉTROPOLE
Le Forum
3 rue Malakoff
38031 Grenoble cedex 01
grenoblealpemetropole.fr

Identité : www.studioplay.fr

### 1. Initial Assessment and Preprocessing

- Create a corpus statistics function to understand document characteristics
- Identify document patterns across your 52 PDFs
- Sample several documents to identify common noise patterns

### 2. Pattern-Based Cleaning

- Remove headers/footers using regex pattern matching
- Eliminate page numbers and watermarks
- Handle hyphenated words split across lines
- Remove citation patterns and references that don't add value
- Clean up table artifacts from OCR process

### 3. Structure Recognition and Segmentation

- Identify document structure (sections, articles, chapters)
- Separate main content from appendices, annexes, and footnotes
- Correct paragraph boundaries
- Process multi-column text correctly